## WANDS-GAR use the google product taxonomy

Can we map queries and products to a common similarity space - in this case the google product taxonomy?

[Run baseline first](https://colab.research.google.com/drive/1yqSJg-99j1uv62CMKD4UC5ahdB1n7HO5) to save baseline index/sample to your GDrive

## Setup

### Download the WANDS e-commerce search dataset

[WANDS is a dataset from Wayfair](https://github.com/wayfair/WANDS) for search experimentation

In [ ]:
!git clone https://github.com/wayfair/WANDS.git
!ls WANDS/dataset

Cloning into 'WANDS'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 40 (delta 7), reused 23 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 33.32 MiB | 9.42 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Updating files: 100% (19/19), done.
label.csv  product.csv	query.csv


### Install dependencies

* SearchArray - a BM25 index
* Pystemmer - for simple stemming
* openai - for openai API access

In [ ]:
!pip install SearchArray==0.0.70 pystemmer openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
  Created wheel for SearchArray: filename=searcharray-0.0.70-cp310-cp310-linux_x86_64.whl size=2996500 sha256=cad33db2829c9af6840ec4cb84ceadfc52b381265386d0371a68c058e4dccb32
  Stored in directory: /root/.cache/pip/wheels/3c/63/e9/ef67f6a4ef51c9d973678b31130ffae69b3

In [ ]:
import pandas as pd
import numpy as np
import Stemmer
import string
from searcharray import SearchArray
import functools
import pickle
import os

from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


## Tokenization + Search functions

Boring snowball/engnlish tokenizers

In [ ]:
stemmer = Stemmer.Stemmer('english', maxCacheSize=0)

fold_to_ascii = dict([(ord(x), ord(y)) for x, y in zip(u"‘’´“”–-", u"'''\"\"--")])
punct_trans = str.maketrans({key: ' ' for key in string.punctuation})
all_trans = {**fold_to_ascii, **punct_trans}


def stem_word(word):
    return stemmer.stemWord(word)


def snowball_tokenizer(text):
    if type(text) == float:
        return ''
    if text is None:
        return ''
    text = text.translate(all_trans).replace("'", " ")
    split = text.lower().split()
    return [stem_word(token)
            for token in split]


def ws_tokenizer(text):
    if type(text) == float:
        return ''
    if text is None:
        return ''
    text = text.translate(all_trans)
    split = text.lower().split()
    return split

def ndcg_m(results):
    max_dcg = np.sum((2**(np.ones(10) + 1)) / np.log(np.arange(10) + 2))
    results['gain'] = (2**results['grade'] - 1) / np.log(results['rank'] + 1)
    dcgs = results.groupby('query')['gain'].sum()
    dcgs = dcgs.sort_values()
    return dcgs / max_dcg

In [ ]:
def ndcg_m(results):
    max_dcg = np.sum((2**(np.ones(10) + 1)) / np.log(np.arange(10) + 2))
    results['gain'] = (2**results['grade'] - 1) / np.log(results['rank'] + 1)
    dcgs = results.groupby('query')['gain'].sum()
    dcgs = dcgs.sort_values()
    return dcgs / max_dcg

def labeled_results(scores, query_labels, downsample, debug=None, N=10):
    """Filter to top N labeled results and assign grades."""

    labeled_mask = downsample.index.isin(query_labels['product_id'])
    scores = scores[labeled_mask]
    sorted_idx = np.argsort(scores)[::-1][:N]
    results = downsample[labeled_mask].iloc[sorted_idx].copy()
    if debug:
        for key, dbg_score in debug.items():
            term, field = key
            results[f'debug_{field}_{term}'] = dbg_score[labeled_mask][sorted_idx]
        # Collapse debug cols to dict
        debug_cols = [col for col in results.columns if col.startswith('debug_')]
        results['debug'] = results[debug_cols].apply(lambda row: row.dropna().to_dict(), axis=1)

    results['scores'] = scores[sorted_idx]
    query_id = query_labels['query_id'].iloc[0]
    results =  results.merge(labels[labels['query_id'] == query_id][['product_id', 'grade']],
                             how='left',
                             on='product_id')
    results['grade'].fillna(0.001, inplace=True)
    assert len(results[(results['grade'] > 0) & (results['grade'] < 0.1)]) == 0

    return results

def search_term_centric(query,
                        downsample,
                        labels,
                        fields=["product_name_snowball", "product_description_snowball",
                                "llm_title_snowball", "llm_description_snowball"],
                        N=10):
    debug = {}
    scores = np.zeros(len(downsample))
    terms_so_far = set()
    for term in snowball_tokenizer(query):
        if term in terms_so_far:
            continue
        terms_so_far.add(term)
        best_term_match = np.zeros(len(downsample))
        for field in fields:
            field_w_boost = field.split('^')
            if len(field_w_boost) == 1:
                boost = 1
                search_field = field
            elif len(field_w_boost) == 2:
                boost = int(field_w_boost[1])
                search_field = field_w_boost[0]

            term_score = downsample[search_field].array.score(term) * boost
            best_term_match = np.maximum(best_term_match, term_score)
            debug[(term, field)] = term_score

        scores += best_term_match

    query_labels = labels[labels['query'] == query]
    return labeled_results(scores, query_labels, downsample, debug=debug, N=N)


## Load data from baseline notebook

Load data from [baseline tokenization notebook](https://colab.research.google.com/drive/1yqSJg-99j1uv62CMKD4UC5ahdB1n7HO5?authuser=1#scrollTo=_bXaNlNHj8io)

In [ ]:
results_baseline = pd.read_pickle('/content/drive/MyDrive/wands-gar/0.results_baseline.pkl')
downsample = pd.read_pickle('/content/drive/MyDrive/wands-gar/0.downsample.pkl')
test_queries = results_baseline['query'].unique()

test_queries = results_baseline['query'].unique()
test_queries

array(['desk for kids', 'jordanna solid wood rocking', 'oriental vanity',
       'nectar queen mattress',
       'bedroom wall decor floral, multicolored with some teal (prints)',
       '48 in entry table with side by side drawer', 'alyse 8 light',
       'kari 2 piece', 'tufted upholstered bed diamond'], dtype=object)

In [ ]:
queries = pd.read_csv("WANDS/dataset/query.csv",
                       delimiter="\t")
labels = pd.read_csv("WANDS/dataset/label.csv",
                      delimiter="\t")

labels
labels.loc[labels['label'] == 'Exact', 'grade'] = 2
labels.loc[labels['label'] == 'Partial', 'grade'] = 1
labels.loc[labels['label'] == 'Irrelevant', 'grade'] = 0
labels = labels.merge(queries, how='left', on='query_id')
assert len(labels[labels['query'].isin(test_queries)]['query'].unique()) == len(test_queries)
labeled_products = labels[labels['query'].isin(test_queries)]['product_id'].unique()
labels

,id,query_id,product_id,label,grade,query,query_class
0,0,0,25434,Exact,2.0,salon chair,Massage Chairs
1,1,0,12088,Irrelevant,0.0,salon chair,Massage Chairs
2,2,0,42931,Exact,2.0,salon chair,Massage Chairs
3,3,0,2636,Exact,2.0,salon chair,Massage Chairs
4,4,0,42923,Exact,2.0,salon chair,Massage Chairs
...,...,...,...,...,...,...,...
233443,234010,478,15439,Partial,1.0,worn leather office chair,Office Chairs
233444,234011,478,451,Partial,1.0,worn leather office chair,Office Chairs
233445,234012,478,30764,Irrelevant,0.0,worn leather office chair,Office Chairs
233446,234013,478,16796,Partial,1.0,worn leather office chair,Office Chairs


## Proposal - map products+queries to taxonomy

In [ ]:
from getpass import getpass

OPENAI_API_KEY = getpass("Your OpenAI Key")

Your OpenAI Key··········


In [ ]:
import functools
import pickle
import os

CACHE_PATH = "/content/drive/MyDrive/wands-gar/google-tax4"
!mkdir -p {CACHE_PATH}

def memoize_to_file(func):
    cache_file = f"{CACHE_PATH}/llm_cache_mices_{{idx}}.pickle"
    cache_len = 0

    cache = {}
    for filename in os.listdir(CACHE_PATH):
        if filename.endswith(".pickle"):
            print(filename)
            try:
                with open(os.path.join(CACHE_PATH, filename), "rb") as f:
                    pkl = pickle.load(f)
                    first_key = list(pkl.keys())[0]
                    print("first item loaded", first_key)
                    cache.update(pkl)
                    print(cache[first_key])
                    print(f"Loaded {filename} -- {len(pkl)} -- Cache has {len(cache)} products")
            except (EOFError, pickle.UnpicklingError) as e:
                print(f"Filename {filename} failed {e}")
                pass
    print("Cache updated and now ", len(cache))

    cache_len = len(cache)

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key = (args, frozenset(kwargs.items()))
        if key not in cache:
            result = func(*args, **kwargs)
            cache[key] = result
            cache_len=len(cache)
            filename = cache_file.format(idx=cache_len//100)
            with open(filename, "wb") as f:
                pickle.dump(cache, f)
        else:
            result = cache[key]
        return result
    return wrapper

In [ ]:
#@title Google product taxonomy (collapsed)


taxonomy = """
436 - Furniture
554 - Furniture > Baby & Toddler Furniture
6349 - Furniture > Baby & Toddler Furniture > Baby & Toddler Furniture Sets
7068 - Furniture > Baby & Toddler Furniture > Bassinet & Cradle Accessories
6393 - Furniture > Baby & Toddler Furniture > Bassinets & Cradles
558 - Furniture > Baby & Toddler Furniture > Changing Tables
7070 - Furniture > Baby & Toddler Furniture > Crib & Toddler Bed Accessories
7072 - Furniture > Baby & Toddler Furniture > Crib & Toddler Bed Accessories > Crib Bumpers & Liners
7071 - Furniture > Baby & Toddler Furniture > Crib & Toddler Bed Accessories > Crib Conversion Kits
6394 - Furniture > Baby & Toddler Furniture > Cribs & Toddler Beds
6969 - Furniture > Baby & Toddler Furniture > High Chair & Booster Seat Accessories
559 - Furniture > Baby & Toddler Furniture > High Chairs & Booster Seats
6433 - Furniture > Beds & Accessories
4437 - Furniture > Beds & Accessories > Bed & Bed Frame Accessories
505764 - Furniture > Beds & Accessories > Beds & Bed Frames
451 - Furniture > Beds & Accessories > Headboards & Footboards
2720 - Furniture > Beds & Accessories > Mattress Foundations
2696 - Furniture > Beds & Accessories > Mattresses
441 - Furniture > Benches
6850 - Furniture > Benches > Kitchen & Dining Benches
6851 - Furniture > Benches > Storage & Entryway Benches
4241 - Furniture > Benches > Vanity Benches
6356 - Furniture > Cabinets & Storage
4063 - Furniture > Cabinets & Storage > Armoires & Wardrobes
447 - Furniture > Cabinets & Storage > Buffets & Sideboards
448 - Furniture > Cabinets & Storage > China Cabinets & Hutches
4195 - Furniture > Cabinets & Storage > Dressers
463 - Furniture > Cabinets & Storage > File Cabinets
465846 - Furniture > Cabinets & Storage > Ironing Centers
6934 - Furniture > Cabinets & Storage > Kitchen Cabinets
6539 - Furniture > Cabinets & Storage > Magazine Racks
6358 - Furniture > Cabinets & Storage > Media Storage Cabinets & Racks
5938 - Furniture > Cabinets & Storage > Storage Cabinets & Lockers
4205 - Furniture > Cabinets & Storage > Storage Chests
6947 - Furniture > Cabinets & Storage > Storage Chests > Hope Chests
4268 - Furniture > Cabinets & Storage > Storage Chests > Toy Chests
4148 - Furniture > Cabinets & Storage > Vanities
2081 - Furniture > Cabinets & Storage > Vanities > Bathroom Vanities
6360 - Furniture > Cabinets & Storage > Vanities > Bedroom Vanities
6357 - Furniture > Cabinets & Storage > Wine & Liquor Cabinets
5578 - Furniture > Cabinets & Storage > Wine Racks
442 - Furniture > Carts & Islands
453 - Furniture > Carts & Islands > Kitchen & Dining Carts
6374 - Furniture > Carts & Islands > Kitchen Islands
7248 - Furniture > Chair Accessories
8206 - Furniture > Chair Accessories > Hanging Chair Replacement Parts
443 - Furniture > Chairs
6499 - Furniture > Chairs > Arm Chairs, Recliners & Sleeper Chairs
438 - Furniture > Chairs > Bean Bag Chairs
456 - Furniture > Chairs > Chaises
2919 - Furniture > Chairs > Electric Massaging Chairs
500051 - Furniture > Chairs > Floor Chairs
3358 - Furniture > Chairs > Folding Chairs & Stools
6800 - Furniture > Chairs > Gaming Chairs
7197 - Furniture > Chairs > Hanging Chairs
5886 - Furniture > Chairs > Kitchen & Dining Room Chairs
2002 - Furniture > Chairs > Rocking Chairs
6859 - Furniture > Chairs > Slipper Chairs
1463 - Furniture > Chairs > Table & Bar Stools
457 - Furniture > Entertainment Centers & TV Stands
6345 - Furniture > Furniture Sets
500000 - Furniture > Furniture Sets > Bathroom Furniture Sets
6346 - Furniture > Furniture Sets > Bedroom Furniture Sets
6347 - Furniture > Furniture Sets > Kitchen & Dining Furniture Sets
6348 - Furniture > Furniture Sets > Living Room Furniture Sets
6860 - Furniture > Futon Frames
2786 - Furniture > Futon Pads
450 - Furniture > Futons
6362 - Furniture > Office Furniture
4191 - Furniture > Office Furniture > Desks
2045 - Furniture > Office Furniture > Office Chairs
500061 - Furniture > Office Furniture > Office Furniture Sets
6363 - Furniture > Office Furniture > Workspace Tables
2242 - Furniture > Office Furniture > Workspace Tables > Art & Drafting Tables
4317 - Furniture > Office Furniture > Workspace Tables > Conference Room Tables
6908 - Furniture > Office Furniture > Workstations & Cubicles
503765 - Furniture > Office Furniture Accessories
503766 - Furniture > Office Furniture Accessories > Desk Parts & Accessories
7559 - Furniture > Office Furniture Accessories > Office Chair Accessories
6909 - Furniture > Office Furniture Accessories > Workstation & Cubicle Accessories
458 - Furniture > Ottomans
4299 - Furniture > Outdoor Furniture
6892 - Furniture > Outdoor Furniture > Outdoor Beds
6367 - Furniture > Outdoor Furniture > Outdoor Furniture Sets
6822 - Furniture > Outdoor Furniture > Outdoor Ottomans
6368 - Furniture > Outdoor Furniture > Outdoor Seating
5044 - Furniture > Outdoor Furniture > Outdoor Seating > Outdoor Benches
6828 - Furniture > Outdoor Furniture > Outdoor Seating > Outdoor Chairs
500111 - Furniture > Outdoor Furniture > Outdoor Seating > Outdoor Sectional Sofa Units
4513 - Furniture > Outdoor Furniture > Outdoor Seating > Outdoor Sofas
4105 - Furniture > Outdoor Furniture > Outdoor Seating > Sunloungers
7310 - Furniture > Outdoor Furniture > Outdoor Storage Boxes
2684 - Furniture > Outdoor Furniture > Outdoor Tables
6963 - Furniture > Outdoor Furniture Accessories
6964 - Furniture > Outdoor Furniture Accessories > Outdoor Furniture Covers
6915 - Furniture > Room Divider Accessories
4163 - Furniture > Room Dividers
464 - Furniture > Shelving
465 - Furniture > Shelving > Bookcases & Standing Shelves
6372 - Furniture > Shelving > Wall Shelves & Ledges
8023 - Furniture > Shelving Accessories
8024 - Furniture > Shelving Accessories > Replacement Shelves
7212 - Furniture > Sofa Accessories
7213 - Furniture > Sofa Accessories > Chair & Sofa Supports
500064 - Furniture > Sofa Accessories > Sectional Sofa Units
460 - Furniture > Sofas
6913 - Furniture > Table Accessories
6911 - Furniture > Table Accessories > Table Legs
6910 - Furniture > Table Accessories > Table Tops
6392 - Furniture > Tables
6369 - Furniture > Tables > Accent Tables
1395 - Furniture > Tables > Accent Tables > Coffee Tables
1549 - Furniture > Tables > Accent Tables > End Tables
1602 - Furniture > Tables > Accent Tables > Sofa Tables
6351 - Furniture > Tables > Activity Tables
4080 - Furniture > Tables > Folding Tables
4355 - Furniture > Tables > Kitchen & Dining Room Tables
4484 - Furniture > Tables > Kotatsu
462 - Furniture > Tables > Nightstands
2693 - Furniture > Tables > Poker & Game Tables
5121 - Furniture > Tables > Sewing Machine Tables
574 - Home & Garden > Bathroom Accessories
575 - Home & Garden > Bathroom Accessories > Bath Caddies
577 - Home & Garden > Bathroom Accessories > Bath Mats & Rugs
4366 - Home & Garden > Bathroom Accessories > Bath Pillows
7093 - Home & Garden > Bathroom Accessories > Bathroom Accessory Mounts
6858 - Home & Garden > Bathroom Accessories > Bathroom Accessory Sets
579 - Home & Garden > Bathroom Accessories > Facial Tissue Holders
8016 - Home & Garden > Bathroom Accessories > Hand Dryer Accessories
5141 - Home & Garden > Bathroom Accessories > Hand Dryers
2418 - Home & Garden > Bathroom Accessories > Medicine Cabinets
2034 - Home & Garden > Bathroom Accessories > Robe Hooks
8114 - Home & Garden > Bathroom Accessories > Safety Grab Bars
578 - Home & Garden > Bathroom Accessories > Shower Curtain Rings
580 - Home & Garden > Bathroom Accessories > Shower Curtains
1962 - Home & Garden > Bathroom Accessories > Shower Rods
4971 - Home & Garden > Bathroom Accessories > Soap & Lotion Dispensers
582 - Home & Garden > Bathroom Accessories > Soap Dishes & Holders
7509 - Home & Garden > Bathroom Accessories > Toilet Brush Replacement Heads
583 - Home & Garden > Bathroom Accessories > Toilet Brushes & Holders
584 - Home & Garden > Bathroom Accessories > Toilet Paper Holders
585 - Home & Garden > Bathroom Accessories > Toothbrush Holders
586 - Home & Garden > Bathroom Accessories > Towel Racks & Holders
359 - Home & Garden > Business & Home Security
5491 - Home & Garden > Business & Home Security > Dummy Surveillance Cameras
3873 - Home & Garden > Business & Home Security > Home Alarm Systems
2161 - Home & Garden > Business & Home Security > Motion Sensors
500025 - Home & Garden > Business & Home Security > Safety & Security Mirrors
363 - Home & Garden > Business & Home Security > Security Lights
364 - Home & Garden > Business & Home Security > Security Monitors & Recorders
499865 - Home & Garden > Business & Home Security > Security Safe Accessories
3819 - Home & Garden > Business & Home Security > Security Safes
365 - Home & Garden > Business & Home Security > Security System Sensors
696 - Home & Garden > Decor
572 - Home & Garden > Decor > Address Signs
6265 - Home & Garden > Decor > Artificial Flora
6266 - Home & Garden > Decor > Artificial Food
9 - Home & Garden > Decor > Artwork
500045 - Home & Garden > Decor > Artwork > Decorative Tapestries
500044 - Home & Garden > Decor > Artwork > Posters, Prints, & Visual Artwork
11 - Home & Garden > Decor > Artwork > Sculptures & Statues
4456 - Home & Garden > Decor > Backrest Pillows
573 - Home & Garden > Decor > Baskets
5521 - Home & Garden > Decor > Bird & Wildlife Feeder Accessories
6993 - Home & Garden > Decor > Bird & Wildlife Feeders
698 - Home & Garden > Decor > Bird & Wildlife Feeders > Bird Feeders
6995 - Home & Garden > Decor > Bird & Wildlife Feeders > Butterfly Feeders
6994 - Home & Garden > Decor > Bird & Wildlife Feeders > Squirrel Feeders
230911 - Home & Garden > Decor > Bird & Wildlife House Accessories
500078 - Home & Garden > Decor > Bird & Wildlife Houses
500079 - Home & Garden > Decor > Bird & Wildlife Houses > Bat Houses
699 - Home & Garden > Decor > Bird & Wildlife Houses > Birdhouses
500080 - Home & Garden > Decor > Bird & Wildlife Houses > Butterfly Houses
697 - Home & Garden > Decor > Bird Baths
587 - Home & Garden > Decor > Bookends
7380 - Home & Garden > Decor > Cardboard Cutouts
4453 - Home & Garden > Decor > Chair & Sofa Cushions
505827 - Home & Garden > Decor > Clock Parts
3890 - Home & Garden > Decor > Clocks
4546 - Home & Garden > Decor > Clocks > Alarm Clocks
6912 - Home & Garden > Decor > Clocks > Desk & Shelf Clocks
3696 - Home & Garden > Decor > Clocks > Floor & Grandfather Clocks
3840 - Home & Garden > Decor > Clocks > Wall Clocks
5708 - Home & Garden > Decor > Coat & Hat Racks
7206 - Home & Garden > Decor > Decorative Bells
6317 - Home & Garden > Decor > Decorative Bottles
6457 - Home & Garden > Decor > Decorative Bowls
7113 - Home & Garden > Decor > Decorative Jars
708 - Home & Garden > Decor > Decorative Plaques
5875 - Home & Garden > Decor > Decorative Plates
6456 - Home & Garden > Decor > Decorative Trays
2675 - Home & Garden > Decor > Door Mats
7172 - Home & Garden > Decor > Dreamcatchers
6936 - Home & Garden > Decor > Dried Flowers
6935 - Home & Garden > Decor > Ecospheres
5609 - Home & Garden > Decor > Figurines
7422 - Home & Garden > Decor > Finials
7419 - Home & Garden > Decor > Flag & Windsock Accessories
7420 - Home & Garden > Decor > Flag & Windsock Accessories > Flag & Windsock Pole Lights
503010 - Home & Garden > Decor > Flag & Windsock Accessories > Flag & Windsock Pole Mounting Hardware & Kits
7421 - Home & Garden > Decor > Flag & Windsock Accessories > Flag & Windsock Poles
701 - Home & Garden > Decor > Flags & Windsocks
4770 - Home & Garden > Decor > Flameless Candles
702 - Home & Garden > Decor > Fountains & Ponds
2921 - Home & Garden > Decor > Fountains & Ponds > Fountain & Pond Accessories
6763 - Home & Garden > Decor > Fountains & Ponds > Fountains & Waterfalls
2763 - Home & Garden > Decor > Fountains & Ponds > Ponds
704 - Home & Garden > Decor > Garden & Stepping Stones
499693 - Home & Garden > Decor > Growth Charts
3221 - Home & Garden > Decor > Home Decor Decals
500121 - Home & Garden > Decor > Home Fragrance Accessories
6336 - Home & Garden > Decor > Home Fragrance Accessories > Candle & Oil Warmers
2784 - Home & Garden > Decor > Home Fragrance Accessories > Candle Holders
500122 - Home & Garden > Decor > Home Fragrance Accessories > Candle Snuffers
4741 - Home & Garden > Decor > Home Fragrance Accessories > Incense Holders
592 - Home & Garden > Decor > Home Fragrances
3898 - Home & Garden > Decor > Home Fragrances > Air Fresheners
588 - Home & Garden > Decor > Home Fragrances > Candles
5847 - Home & Garden > Decor > Home Fragrances > Fragrance Oil
3686 - Home & Garden > Decor > Home Fragrances > Incense
4740 - Home & Garden > Decor > Home Fragrances > Potpourri
6767 - Home & Garden > Decor > Home Fragrances > Wax Tarts
503000 - Home & Garden > Decor > Hourglasses
7382 - Home & Garden > Decor > House Numbers & Letters
6547 - Home & Garden > Decor > Lawn Ornaments & Garden Sculptures
7436 - Home & Garden > Decor > Mail Slots
6333 - Home & Garden > Decor > Mailbox Accessories
7177 - Home & Garden > Decor > Mailbox Accessories > Mailbox Covers
7052 - Home & Garden > Decor > Mailbox Accessories > Mailbox Enclosures
7176 - Home & Garden > Decor > Mailbox Accessories > Mailbox Flags
6334 - Home & Garden > Decor > Mailbox Accessories > Mailbox Posts
7339 - Home & Garden > Decor > Mailbox Accessories > Mailbox Replacement Doors
706 - Home & Garden > Decor > Mailboxes
595 - Home & Garden > Decor > Mirrors
3473 - Home & Garden > Decor > Music Boxes
6324 - Home & Garden > Decor > Napkin Rings
5885 - Home & Garden > Decor > Novelty Signs
6927 - Home & Garden > Decor > Ottoman Cushions
597 - Home & Garden > Decor > Picture Frames
4295 - Home & Garden > Decor > Piggy Banks & Money Jars
709 - Home & Garden > Decor > Rain Chains
710 - Home & Garden > Decor > Rain Gauges
5876 - Home & Garden > Decor > Refrigerator Magnets
598 - Home & Garden > Decor > Rugs
596 - Home & Garden > Decor > Seasonal & Holiday Decorations
5359 - Home & Garden > Decor > Seasonal & Holiday Decorations > Advent Calendars
5504 - Home & Garden > Decor > Seasonal & Holiday Decorations > Christmas Tree Skirts
6603 - Home & Garden > Decor > Seasonal & Holiday Decorations > Christmas Tree Stands
499805 - Home & Garden > Decor > Seasonal & Holiday Decorations > Easter Egg Decorating Kits
6532 - Home & Garden > Decor > Seasonal & Holiday Decorations > Holiday Ornament Displays & Stands
499804 - Home & Garden > Decor > Seasonal & Holiday Decorations > Holiday Ornament Hooks
3144 - Home & Garden > Decor > Seasonal & Holiday Decorations > Holiday Ornaments
5990 - Home & Garden > Decor > Seasonal & Holiday Decorations > Holiday Stocking Hangers
5991 - Home & Garden > Decor > Seasonal & Holiday Decorations > Holiday Stockings
5930 - Home & Garden > Decor > Seasonal & Holiday Decorations > Japanese Traditional Dolls
6531 - Home & Garden > Decor > Seasonal & Holiday Decorations > Nativity Sets
505809 - Home & Garden > Decor > Seasonal & Holiday Decorations > Seasonal Village Sets & Accessories
5922 - Home & Garden > Decor > Shadow Boxes
599 - Home & Garden > Decor > Slipcovers
6535 - Home & Garden > Decor > Snow Globes
7173 - Home & Garden > Decor > Suncatchers
711 - Home & Garden > Decor > Sundials
4454 - Home & Garden > Decor > Throw Pillows
4233 - Home & Garden > Decor > Trunks
6424 - Home & Garden > Decor > Vase Fillers & Table Scatters
602 - Home & Garden > Decor > Vases
2334 - Home & Garden > Decor > Wallpaper
712 - Home & Garden > Decor > Weather Vanes & Roof Decor
714 - Home & Garden > Decor > Wind Chimes
2839 - Home & Garden > Decor > Wind Wheels & Spinners
6530 - Home & Garden > Decor > Window Magnets
6254 - Home & Garden > Decor > Window Treatment Accessories
6256 - Home & Garden > Decor > Window Treatment Accessories > Curtain & Drape Rings
6257 - Home & Garden > Decor > Window Treatment Accessories > Curtain & Drape Rods
6255 - Home & Garden > Decor > Window Treatment Accessories > Curtain Holdbacks & Tassels
8042 - Home & Garden > Decor > Window Treatment Accessories > Window Treatment Replacement Parts
603 - Home & Garden > Decor > Window Treatments
2882 - Home & Garden > Decor > Window Treatments > Curtains & Drapes
6492 - Home & Garden > Decor > Window Treatments > Stained Glass Panels
2885 - Home & Garden > Decor > Window Treatments > Window Blinds & Shades
5989 - Home & Garden > Decor > Window Treatments > Window Films
4375 - Home & Garden > Decor > Window Treatments > Window Screens
2621 - Home & Garden > Decor > Window Treatments > Window Valances & Cornices
3262 - Home & Garden > Decor > World Globes
6267 - Home & Garden > Decor > Wreaths & Garlands
638 - Home & Garden > Kitchen & Dining
2901 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories
3489 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Breadmaker Accessories
3988 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories
6888 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Coffee Decanter Warmers
3239 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Coffee Decanters
4500 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Coffee Filter Baskets
3450 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Coffee Filters
4786 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Coffee Grinder Accessories
734 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Coffee Grinders
503736 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Coffee Maker & Espresso Machine Replacement Parts
5065 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Coffee Maker Water Filters
5066 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Frothing Pitchers
3838 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Coffee Maker & Espresso Machine Accessories > Portafilters
500004 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Cooktop, Oven & Range Accessories
5076 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Cotton Candy Machine Accessories
3954 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Deep Fryer Accessories
3443 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Dishwasher Parts & Accessories
500066 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Electric Kettle Accessories
7355 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Electric Skillet & Wok Accessories
6944 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Fondue Set Accessories
503725 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Fondue Set Accessories > Cooking Gel Fuels
6945 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Fondue Set Accessories > Fondue Forks
6946 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Fondue Set Accessories > Fondue Pot Stands
4653 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Food Dehydrator Accessories
4655 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Food Dehydrator Accessories > Food Dehydrator Sheets
4654 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Food Dehydrator Accessories > Food Dehydrator Trays
4763 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Food Grinder Accessories
505765 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Food Mixer & Blender Accessories
7570 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Freezer Accessories
6747 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Garbage Disposal Accessories
4674 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Ice Cream Maker Accessories
4675 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Ice Cream Maker Accessories > Ice Cream Maker Freezer Bowls
5042 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Ice Crusher & Shaver Accessories
7187 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Ice Maker Accessories
4519 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Juicer Accessories
1334 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Microwave Oven Accessories
3684 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories
5694 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories > Charcoal Briquettes
7540 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories > Charcoal Chimneys
5670 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories > Outdoor Grill Carts
3855 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories > Outdoor Grill Covers
3382 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories > Outdoor Grill Racks & Toppers
505667 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories > Outdoor Grill Replacement Parts
4560 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories > Outdoor Grill Spits & Baskets
5672 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories > Outdoor Grilling Planks
5671 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Outdoor Grill Accessories > Smoking Chips & Pellets
2540 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Pasta Maker Accessories
5075 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Popcorn Maker Accessories
7006 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Portable Cooking Stove Accessories
8087 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Range Hood Accessories
3848 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Refrigerator Accessories
502989 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Soda Maker Accessories
8051 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Steam Table Accessories
8052 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Steam Table Accessories > Steam Table Pan Covers
8053 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Steam Table Accessories > Steam Table Pans
7444 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Toaster Accessories
3523 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Vacuum Sealer Accessories
3124 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Vacuum Sealer Accessories > Vacuum Sealer Bags
499996 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Waffle Iron Accessories
7118 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Water Cooler Accessories
7119 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Water Cooler Accessories > Water Cooler Bottles
8106 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Wine Fridge Accessories
5570 - Home & Garden > Kitchen & Dining > Kitchen Appliance Accessories > Yogurt Maker Accessories
730 - Home & Garden > Kitchen & Dining > Kitchen Appliances
5287 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Beverage Warmers
732 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Breadmakers
5090 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Chocolate Tempering Machines
736 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Coffee Makers & Espresso Machines
1388 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Coffee Makers & Espresso Machines > Drip Coffee Makers
1647 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Coffee Makers & Espresso Machines > Electric & Stovetop Espresso Pots
2422 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Coffee Makers & Espresso Machines > Espresso Machines
1557 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Coffee Makers & Espresso Machines > French Presses
2247 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Coffee Makers & Espresso Machines > Percolators
5286 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Coffee Makers & Espresso Machines > Vacuum Coffee Makers
679 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Cooktops
3319 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Cotton Candy Machines
738 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Deep Fryers
3181 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Deli Slicers
680 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Dishwashers
7165 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Electric Griddles & Grills
751 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Electric Kettles
4421 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Electric Skillets & Woks
4720 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Fondue Pots & Sets
4532 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Cookers & Steamers
739 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Cookers & Steamers > Egg Cookers
760 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Cookers & Steamers > Food Steamers
757 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Cookers & Steamers > Rice Cookers
737 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Cookers & Steamers > Slow Cookers
6523 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Cookers & Steamers > Thermal Cookers
6279 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Cookers & Steamers > Water Ovens
743 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Dehydrators
744 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Grinders & Mills
505666 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Mixers & Blenders
687 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Smokers
5103 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Warmers
6548 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Warmers > Chafing Dishes
5349 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Warmers > Food Heat Lamps
504633 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Warmers > Rice Keepers
4292 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Food Warmers > Steam Tables
681 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Freezers
5156 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Frozen Drink Makers
610 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Garbage Disposals
6524 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Gas Griddles
6543 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Hot Drink Makers
747 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Hot Plates
748 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Ice Cream Makers
749 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Ice Crushers & Shavers
4161 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Ice Makers
750 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Juicers
752 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Knife Sharpeners
753 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Microwave Ovens
3526 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Milk Frothers & Steamers
4482 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Mochi Makers
2985 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Outdoor Grills
683 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Ovens
755 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Pasta Makers
756 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Popcorn Makers
1015 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Portable Cooking Stoves
684 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Range Hoods
685 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Ranges
686 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Refrigerators
4495 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Roaster Ovens & Rotisseries
5577 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Soda Makers
5057 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Soy Milk Makers
4528 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Tea Makers
5289 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills
761 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Countertop & Toaster Ovens
6819 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Donut Makers
5318 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Muffin & Cupcake Makers
6278 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Pizza Makers & Ovens
5291 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Pizzelle Makers
6516 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Pretzel Makers
759 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Sandwich Makers
762 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Toasters
5292 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Tortilla & Flatbread Makers
764 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Toasters & Grills > Waffle Irons
688 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Trash Compactors
763 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Vacuum Sealers
3293 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Water Coolers
765 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Water Filters
4539 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Wine Fridges
766 - Home & Garden > Kitchen & Dining > Kitchen Appliances > Yogurt Makers
668 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils
639 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Aprons
3768 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Baking Peels
3347 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Basters
3430 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Basting Brushes
7149 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Beverage Dispensers
4630 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cake Decorating Supplies
6408 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cake Servers
4247 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Can Crushers
733 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Can Openers
5078 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Carving Forks
6522 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Channel Knives
653 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Colanders & Strainers
4777 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Condiment Dispensers
3850 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cookie Cutters
6342 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cookie Presses
7331 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cooking Thermometer Accessories
3091 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cooking Thermometers
3713 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cooking Timers
5928 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cooking Torches
3835 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cooling Racks
666 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Cutting Boards
3268 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Dish Racks & Drain Boards
6723 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Dough Wheels
6411 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Electric Knife Accessories
6412 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Electric Knife Accessories > Electric Knife Replacement Blades
741 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Electric Knives
5370 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Flour Sifters
505316 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food & Drink Stencils
3381 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food Crackers
3586 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food Crackers > Lobster & Crab Crackers
3685 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food Crackers > Nutcrackers
4214 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food Crackers > Nutcrackers > Decorative Nutcrackers
3723 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food Dispensers
3156 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food Graters & Zesters
3521 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food Peelers & Corers
7329 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food Steaming Bags
6554 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Food Sticks & Skewers
503005 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Funnels
3385 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Garlic Presses
6787 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Gelatin Molds
4746 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Ice Cube Trays
7485 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Jerky Guns
665 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Knives
8006 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Molds
2948 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers
6480 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Can Organizers
3479 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Drinkware Holders
6487 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Kitchen Cabinet Organizers
3177 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Kitchen Counter & Beverage Station Organizers
8012 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Kitchen Utensil Holders & Racks
5157 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Knife Blocks & Holders
3072 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Napkin Holders & Dispensers
3061 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Paper Towel Holders & Dispensers
3845 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Pot Racks
2344 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Spice Organizers
5059 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Straw Holders & Dispensers
6415 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Sugar Caddies
4322 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Toothpick Holders & Dispensers
3831 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Organizers > Utensil & Flatware Trays
3256 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Scrapers
3419 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Scrapers > Bench Scrapers
3086 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Scrapers > Bowl Scrapers
3633 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Scrapers > Grill Scrapers
5251 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Shears
3206 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Slicers
4765 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Kitchen Utensil Sets
3620 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Ladles
3294 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Mashers
3475 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Measuring Cups & Spoons
3248 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Meat Tenderizers
4530 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Mixing Bowls
3999 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Mortars & Pestles
6526 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Oil & Vinegar Dispensers
4771 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Oven Bags
670 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Oven Mitts & Pot Holders
6749 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Pasta Molds & Stamps
4332 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Pastry Blenders
4708 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Pastry Cloths
7365 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Pizza Cutter Accessories
3421 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Pizza Cutters
5109 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Ricers
4705 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Rolling Pin Accessories
4706 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Rolling Pin Accessories > Rolling Pin Covers & Sleeves
4707 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Rolling Pin Accessories > Rolling Pin Rings
3467 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Rolling Pins
6497 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Salad Dressing Mixers & Shakers
3914 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Salad Spinners
3175 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Scoops
3202 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Scoops > Ice Cream Scoops
3708 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Scoops > Ice Scoops
3258 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Scoops > Melon Ballers
502966 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Scoops > Popcorn & French Fry Scoops
6746 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Sink Caddies
5080 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Sink Mats & Grids
6388 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Slotted Spoons
3196 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Spatulas
4788 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Spice Grinder Accessories
4762 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Spice Grinders
4334 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Spoon Rests
6974 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Sugar Dispensers
7247 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Sushi Mats
4559 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Tea Strainers
4005 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Tongs
3597 - Home & Garden > Kitchen & Dining > Kitchen Tools & Utensils > Whisks
8161 - Home & Garden > Kitchen & Dining > Prefabricated Kitchens & Kitchenettes
672 - Home & Garden > Kitchen & Dining > Tableware
6740 - Home & Garden > Kitchen & Dining > Tableware > Coffee & Tea Sets
652 - Home & Garden > Kitchen & Dining > Tableware > Coffee Servers & Tea Pots
673 - Home & Garden > Kitchen & Dining > Tableware > Dinnerware
3498 - Home & Garden > Kitchen & Dining > Tableware > Dinnerware > Bowls
5537 - Home & Garden > Kitchen & Dining > Tableware > Dinnerware > Dinnerware Sets
3553 - Home & Garden > Kitchen & Dining > Tableware > Dinnerware > Plates
674 - Home & Garden > Kitchen & Dining > Tableware > Drinkware
7568 - Home & Garden > Kitchen & Dining > Tableware > Drinkware > Beer Glasses
6049 - Home & Garden > Kitchen & Dining > Tableware > Drinkware > Coffee & Tea Cups
6051 - Home & Garden > Kitchen & Dining > Tableware > Drinkware > Coffee & Tea Saucers
6958 - Home & Garden > Kitchen & Dining > Tableware > Drinkware > Drinkware Sets
2169 - Home & Garden > Kitchen & Dining > Tableware > Drinkware > Mugs
2694 - Home & Garden > Kitchen & Dining > Tableware > Drinkware > Shot Glasses
2712 - Home & Garden > Kitchen & Dining > Tableware > Drinkware > Stemware
2951 - Home & Garden > Kitchen & Dining > Tableware > Drinkware > Tumblers
675 - Home & Garden > Kitchen & Dining > Tableware > Flatware
6439 - Home & Garden > Kitchen & Dining > Tableware > Flatware > Chopstick Accessories
3699 - Home & Garden > Kitchen & Dining > Tableware > Flatware > Chopsticks
5647 - Home & Garden > Kitchen & Dining > Tableware > Flatware > Flatware Sets
4015 - Home & Garden > Kitchen & Dining > Tableware > Flatware > Forks
3939 - Home & Garden > Kitchen & Dining > Tableware > Flatware > Spoons
3844 - Home & Garden > Kitchen & Dining > Tableware > Flatware > Table Knives
676 - Home & Garden > Kitchen & Dining > Tableware > Salt & Pepper Shakers
4026 - Home & Garden > Kitchen & Dining > Tableware > Serveware
6086 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Butter Dishes
5135 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Cake Boards
4372 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Cake Stands
7550 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Egg Cups
3703 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Gravy Boats
4735 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Punch Bowls
3330 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Serving Pitchers & Carafes
3802 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Serving Platters
4009 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Serving Trays
3373 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Sugar Bowls & Creamers
3941 - Home & Garden > Kitchen & Dining > Tableware > Serveware > Tureens
6425 - Home & Garden > Kitchen & Dining > Tableware > Serveware Accessories
6434 - Home & Garden > Kitchen & Dining > Tableware > Serveware Accessories > Punch Bowl Stands
6427 - Home & Garden > Kitchen & Dining > Tableware > Serveware Accessories > Tureen Lids
6426 - Home & Garden > Kitchen & Dining > Tableware > Serveware Accessories > Tureen Stands
8046 - Home & Garden > Kitchen & Dining > Tableware > Tablecloth Clips & Weights
677 - Home & Garden > Kitchen & Dining > Tableware > Trivets
689 - Home & Garden > Lawn & Garden
2962 - Home & Garden > Lawn & Garden > Gardening
4085 - Home & Garden > Lawn & Garden > Gardening > Composting
690 - Home & Garden > Lawn & Garden > Gardening > Composting > Compost
6840 - Home & Garden > Lawn & Garden > Gardening > Composting > Compost Aerators
6436 - Home & Garden > Lawn & Garden > Gardening > Composting > Composters
691 - Home & Garden > Lawn & Garden > Gardening > Disease Control
113 - Home & Garden > Lawn & Garden > Gardening > Fertilizers
500033 - Home & Garden > Lawn & Garden > Gardening > Garden Pot Saucers & Trays
5632 - Home & Garden > Lawn & Garden > Gardening > Gardening Accessories
503756 - Home & Garden > Lawn & Garden > Gardening > Gardening Accessories > Gardening Scooters, Seats & Kneelers
5633 - Home & Garden > Lawn & Garden > Gardening > Gardening Accessories > Gardening Totes
7184 - Home & Garden > Lawn & Garden > Gardening > Gardening Accessories > Potting Benches
505326 - Home & Garden > Lawn & Garden > Gardening > Gardening Tool Accessories
505322 - Home & Garden > Lawn & Garden > Gardening > Gardening Tool Accessories > Gardening Tool Handles
505321 - Home & Garden > Lawn & Garden > Gardening > Gardening Tool Accessories > Gardening Tool Heads
4972 - Home & Garden > Lawn & Garden > Gardening > Gardening Tool Accessories > Wheelbarrow Parts
3173 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools
7537 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Bulb Planting Tools
4000 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Cultivating Tools
3071 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Gardening Forks
505292 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Gardening Sickles & Machetes
3644 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Gardening Trowels
1967 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Lawn & Garden Sprayers
499922 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Lawn Rollers
6967 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Pruning Saws
3841 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Pruning Shears
3388 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Rakes
2147 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Shovels & Spades
3828 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Spreaders
3616 - Home & Garden > Lawn & Garden > Gardening > Gardening Tools > Wheelbarrows
693 - Home & Garden > Lawn & Garden > Gardening > Greenhouses
3103 - Home & Garden > Lawn & Garden > Gardening > Herbicides
6381 - Home & Garden > Lawn & Garden > Gardening > Landscape Fabric
6413 - Home & Garden > Lawn & Garden > Gardening > Landscape Fabric Accessories
6422 - Home & Garden > Lawn & Garden > Gardening > Landscape Fabric Accessories > Landscape Fabric Staples & Pins
6421 - Home & Garden > Lawn & Garden > Gardening > Landscape Fabric Accessories > Landscape Fabric Tape
2988 - Home & Garden > Lawn & Garden > Gardening > Mulch
499894 - Home & Garden > Lawn & Garden > Gardening > Plant Cages & Supports
6428 - Home & Garden > Lawn & Garden > Gardening > Plant Stands
499962 - Home & Garden > Lawn & Garden > Gardening > Pot & Planter Liners
721 - Home & Garden > Lawn & Garden > Gardening > Pots & Planters
6834 - Home & Garden > Lawn & Garden > Gardening > Rain Barrels
1794 - Home & Garden > Lawn & Garden > Gardening > Sands & Soils
543677 - Home & Garden > Lawn & Garden > Gardening > Sands & Soils > Sand
543678 - Home & Garden > Lawn & Garden > Gardening > Sands & Soils > Soil
2918 - Home & Garden > Lawn & Garden > Outdoor Living
499908 - Home & Garden > Lawn & Garden > Outdoor Living > Awning Accessories
499907 - Home & Garden > Lawn & Garden > Outdoor Living > Awnings
6737 - Home & Garden > Lawn & Garden > Outdoor Living > Hammock Parts & Accessories
717 - Home & Garden > Lawn & Garden > Outdoor Living > Hammocks
5910 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Blankets
5911 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Blankets > Beach Mats
5913 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Blankets > Picnic Blankets
5912 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Blankets > Poncho Liners
2613 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures
716 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Canopies & Gazebos
6105 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Canopy & Gazebo Accessories
6107 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Canopy & Gazebo Accessories > Canopy & Gazebo Enclosure Kits
6106 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Canopy & Gazebo Accessories > Canopy & Gazebo Frames
6108 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Canopy & Gazebo Accessories > Canopy & Gazebo Tops
7423 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Canopy & Gazebo Accessories > Canopy Poles
7424 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Canopy & Gazebo Accessories > Canopy Weights
703 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Garden Arches, Trellises, Arbors & Pergolas
700 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Garden Bridges
720 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Structures > Sheds, Garages & Carports
6751 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Umbrella & Sunshade Accessories
7108 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Umbrella & Sunshade Accessories > Outdoor Umbrella & Sunshade Fabric
5493 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Umbrella & Sunshade Accessories > Outdoor Umbrella Bases
7107 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Umbrella & Sunshade Accessories > Outdoor Umbrella Covers
499948 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Umbrella & Sunshade Accessories > Outdoor Umbrella Enclosure Kits
8020 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Umbrella & Sunshade Accessories > Outdoor Umbrella Lights
719 - Home & Garden > Lawn & Garden > Outdoor Living > Outdoor Umbrellas & Sunshades
499955 - Home & Garden > Lawn & Garden > Outdoor Living > Porch Swing Accessories
718 - Home & Garden > Lawn & Garden > Outdoor Living > Porch Swings
594 - Home & Garden > Lighting
1436 - Home & Garden > Lighting > Emergency Lighting
500003 - Home & Garden > Lighting > Floating & Submersible Lights
1546 - Home & Garden > Lighting > Flood & Spot Lights
7401 - Home & Garden > Lighting > In-Ground Lights
4636 - Home & Garden > Lighting > Lamps
7400 - Home & Garden > Lighting > Landscape Pathway Lighting
2425 - Home & Garden > Lighting > Light Bulbs
2947 - Home & Garden > Lighting > Light Bulbs > Compact Fluorescent Lamps
2690 - Home & Garden > Lighting > Light Bulbs > Fluorescent Tubes
2944 - Home & Garden > Lighting > Light Bulbs > Incandescent Light Bulbs
3329 - Home & Garden > Lighting > Light Bulbs > LED Light Bulbs
2608 - Home & Garden > Lighting > Light Ropes & Strings
3006 - Home & Garden > Lighting > Lighting Fixtures
2809 - Home & Garden > Lighting > Lighting Fixtures > Cabinet Light Fixtures
2524 - Home & Garden > Lighting > Lighting Fixtures > Ceiling Light Fixtures
2249 - Home & Garden > Lighting > Lighting Fixtures > Chandeliers
6073 - Home & Garden > Lighting > Lighting Fixtures > Wall Light Fixtures
505826 - Home & Garden > Lighting > Night Lights & Ambient Lighting
2370 - Home & Garden > Lighting > Picture Lights
7399 - Home & Garden > Lighting > Tiki Torches & Oil Lamps
6274 - Home & Garden > Lighting > Track Lighting
6272 - Home & Garden > Lighting > Track Lighting > Track Lighting Accessories
4932 - Home & Garden > Lighting > Track Lighting > Track Lighting Fixtures
6273 - Home & Garden > Lighting > Track Lighting > Track Lighting Rails
2956 - Home & Garden > Lighting Accessories
7338 - Home & Garden > Lighting Accessories > Lamp Post Bases
7447 - Home & Garden > Lighting Accessories > Lamp Post Mounts
3185 - Home & Garden > Lighting Accessories > Lamp Shades
3522 - Home & Garden > Lighting Accessories > Lighting Timers
505312 - Home & Garden > Lighting Accessories > Oil Lamp Fuel
4171 - Home & Garden > Linens & Bedding
569 - Home & Garden > Linens & Bedding > Bedding
505803 - Home & Garden > Linens & Bedding > Bedding > Bed Canopies
2314 - Home & Garden > Linens & Bedding > Bedding > Bed Sheets
2974 - Home & Garden > Linens & Bedding > Bedding > Bedskirts
1985 - Home & Garden > Linens & Bedding > Bedding > Blankets
2541 - Home & Garden > Linens & Bedding > Bedding > Duvet Covers
4452 - Home & Garden > Linens & Bedding > Bedding > Mattress Protectors
4420 - Home & Garden > Linens & Bedding > Bedding > Mattress Protectors > Mattress Encasements
2991 - Home & Garden > Linens & Bedding > Bedding > Mattress Protectors > Mattress Pads
1599 - Home & Garden > Linens & Bedding > Bedding > Nap Mats
2927 - Home & Garden > Linens & Bedding > Bedding > Pillowcases & Shams
2700 - Home & Garden > Linens & Bedding > Bedding > Pillows
505287 - Home & Garden > Linens & Bedding > Bedding > Quilts & Comforters
505832 - Home & Garden > Linens & Bedding > Kitchen Linens Sets
601 - Home & Garden > Linens & Bedding > Table Linens
4203 - Home & Garden > Linens & Bedding > Table Linens > Cloth Napkins
4343 - Home & Garden > Linens & Bedding > Table Linens > Doilies
2547 - Home & Garden > Linens & Bedding > Table Linens > Placemats
6325 - Home & Garden > Linens & Bedding > Table Linens > Table Runners
6322 - Home & Garden > Linens & Bedding > Table Linens > Table Skirts
4143 - Home & Garden > Linens & Bedding > Table Linens > Tablecloths
4077 - Home & Garden > Linens & Bedding > Towels
576 - Home & Garden > Linens & Bedding > Towels > Bath Towels & Washcloths
4126 - Home & Garden > Linens & Bedding > Towels > Beach Towels
4257 - Home & Garden > Linens & Bedding > Towels > Kitchen Towels
4358 - Home & Garden > Parasols & Rain Umbrellas
985 - Home & Garden > Plants
5590 - Home & Garden > Plants > Aquatic Plants
984 - Home & Garden > Plants > Flowers
6762 - Home & Garden > Plants > Indoor & Outdoor Plants
543559 - Home & Garden > Plants > Indoor & Outdoor Plants > Bushes & Shrubs
543560 - Home & Garden > Plants > Indoor & Outdoor Plants > Landscaping & Garden Plants
543558 - Home & Garden > Plants > Indoor & Outdoor Plants > Potted Houseplants
505285 - Home & Garden > Plants > Plant & Herb Growing Kits
2802 - Home & Garden > Plants > Seeds
543561 - Home & Garden > Plants > Seeds > Plant & Flower Bulbs
543562 - Home & Garden > Plants > Seeds > Seeds & Seed Tape
1684 - Home & Garden > Plants > Trees
6173 - Home & Garden > Umbrella Sleeves & Cases
2639 - Home & Garden > Wood Stoves"""

In [ ]:
from functools import lru_cache
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

prompt = f"""
I'm going to give you a product description of furniture, you need to tell me the google
product taxonomy the item most likely falls in, using the following subset of the taxonomy.

Prepend your response with "Taxonomy:"

If you are uncertain, please say "Taxonomy:No appropriate classification" if none found.

Here is the taxonomy to use:

{taxonomy}

Some examples:

Product Description: featuring hard iron in a crystal lattice drum shade . this chandelier has clear crystals that gracefully refract light from the five candelabra bulbs for enchanting beauty .
Taxonomy: 2249 - Home & Garden > Lighting > Lighting Fixtures > Chandeliers

Product Description: it has beautiful styling and nailhead trim adding more style to it .
Taxonomy: No appropriate classification

Product Description: A dog leash
Taxonomy: No appropriate classification

Product Description: Desk for kids
Taxonomy: 4191 - Furniture > Office Furniture > Desks

Product Description: beautiful floral prints depicting orchids in a summer glade
Taxonomy: 500044 - Home & Garden > Decor > Artwork > Posters, Prints, & Visual Artwork

Product Description: sweet dreams are a breeze when your little one is snuggled up in wildkin toddler sheet set ! both the top sheet and fitted sheet fit a standard toddler mattres, making this set perfect for transitioning from crib to big kid bed . the
Taxonomy: 2314 - Home & Garden > Linens & Bedding > Bedding > Bed Sheets

Product Description: the teak rocker adirondack chair is another masterpiece of comfort . the deeply contoured back , wide arms and curved seat make this rocker one of their top sellers ! the rocker whispers comfort and relaxation as you sit and rest . enjoy life - order an adirondack rocker today ! this manufacturer 's philosophy about teak furniture is different than most . while all around see designs and styles that minimize the use of teak wood by substituting with aluminum or adding sling fabric material , this manufacturer has gone the other direction . slim and sleek wo n't be found in descriptions of this manufacturer 's furniture . instead , you 'll read masterful , bold and rock solid . everything in the design of these styles has been about making a statement with teak . this manufacturer has added thickness to these parts to give unheard of stability and strength . this manufacturer has also added an extra dimension to provide more room and comfort than any teak designs .
Taxonomy: 6828 - Furniture > Outdoor Furniture > Outdoor Seating > Outdoor Chairs

Product Description: the l-shape office desktop provides ample space for your computer , monitor , and printer . open shelves provide extra space for storing your office supplies and books . the desk also has a wiring hole to help our customers to arrange their disorganized wires . because enough space under the table to let you scratch , therefore the clients can do their work freely in any corner . the instruction also teaches you how to assemble all the parts with the corresponding numbers .
Taxonomy: 4191 - Furniture > Office Furniture > Desks

Product Description: this living room make a statement and create an exciting space for your living room a true style sophisticate , this couch has made its appearance in upscale gentlemen ’ s clubs , victorian homes , and more for centuries ; today . this couch offers a modern take on a timeless classic with its round tufted arms and nail head trim . its mixed hardwood frame is wrapped velvet upholstery with button-tufting along the arms and backrest and wood legs below .
Taxonomy: 6346 - Furniture > Furniture Sets > Living Room Furniture Sets
"""

# Enriched instead of new?
@memoize_to_file
def set_taxonomy(product_description):

    if not product_description:
        return ""

    test = f"""
    Product Description: {product_description}
    """
    # print("API CALL!")

    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
        "role": "system",
        "content": prompt
        },
        {
        "role": "user",
        "content": test
        }
    ],
    temperature=0.1,
    max_tokens=256,
    top_p=1
    )
    resp = response.choices[0].message.content
    resp_split = resp.split("\n")
    title, tags, desc = "", "", ""
    label = "Taxonomy:"
    for i in range(0, len(resp_split)):
        if resp_split[i].startswith(label):
            tags = resp_split[i][len(label)+1:]

    return tags

llm_cache_mices_0.pickle
first item loaded (('featuring hard iron in a crystal lattice drum shade . this chandelier has clear crystals that gracefully refract light from the five candelabra bulbs for enchanting beauty .',), frozenset())
2249 - Home & Garden > Lighting > Lighting Fixtures > Chandeliers
Loaded llm_cache_mices_0.pickle -- 99 -- Cache has 99 products
llm_cache_mices_1.pickle
first item loaded (('featuring hard iron in a crystal lattice drum shade . this chandelier has clear crystals that gracefully refract light from the five candelabra bulbs for enchanting beauty .',), frozenset())
2249 - Home & Garden > Lighting > Lighting Fixtures > Chandeliers
Loaded llm_cache_mices_1.pickle -- 199 -- Cache has 199 products
llm_cache_mices_2.pickle
first item loaded (('featuring hard iron in a crystal lattice drum shade . this chandelier has clear crystals that gracefully refract light from the five candelabra bulbs for enchanting beauty .',), frozenset())
2249 - Home & Garden > Lighti

In [ ]:
set_taxonomy( "featuring hard iron in a crystal lattice drum shade . this chandelier has clear crystals that gracefully refract light from the five candelabra bulbs for enchanting beauty .")

'2249 - Home & Garden > Lighting > Lighting Fixtures > Chandeliers'

In [ ]:
set_taxonomy( "the l-shape office desktop provides ample space for your computer , monitor , and printer . open shelves provide extra space for storing your office supplies and books . the desk also has a wiring hole to help our customers to arrange their disorganized wires . because enough space under the table to let you scratch , therefore the clients can do their work freely in any corner . the instruction also teaches you how to assemble all the parts with the corresponding numbers .")

'4191 - Furniture > Office Furniture > Desks'

In [ ]:
set_taxonomy("office desk for teenager")

'4191 - Furniture > Office Furniture > Desks'

In [ ]:
set_taxonomy( "it has beautiful styling and nailhead trim adding more style to it .")

'No appropriate classification'

In [ ]:
set_taxonomy( "posters of he-man fighting the bad guys, perfect to decorate a dorm room")

'500044 - Home & Garden > Decor > Artwork > Posters, Prints, & Visual Artwork'

In [ ]:
set_taxonomy("impart a traditional appeal to your bedroom with the inclusion of this traditional bed . constructed from a combination of solid wood and veneer it features a leatherette padded scrolled headboard with button tufting . the headboard and sleigh-type footboard display nailhead trim adding more style to it .")

'505764 - Furniture > Beds & Accessories > Beds & Bed Frames'

In [ ]:
set_taxonomy("""this gas pocket bike is ideal for providing driveway and
parking lot commuting entertainment for kids and teenagers 13 years and older .
gas pocket bike comes with 10 inch large pneumatic tires for easy balance and
support.""")

'No appropriate classification'

In [ ]:
set_taxonomy("""minimalism in this curated writing desk . crafted of
100 % mango wood , this table sports a light stain to showcase its natural
exotic grain , while a hand-scraped finish creates a distressed , reclaimed
aesthetic . its clean-lined rectangular desktop provides a perfect platform for
your laptop and accessories , while its streamlined trestle base includes a
stretcher for added support . measuring 29.5 '' h x 47 '' w x 18.5 '' d overall
, this desk accommodates chairs with seat heights of 24 '' to 25 ''""")

'4191 - Furniture > Office Furniture > Desks'

In [ ]:
set_taxonomy("""nothing makes your little one feel like a big kid like their own chair . this
petite design features traditional styling in a smaller-scale format ! it 's
made in the usa , and showcases a curved backrest with winged arms , and is
founded atop four splayed , wooden legs . plus , it 's upholstered with gray and
white fabric that showcases a pattern of peacocks , birds , and horses , for a
whimsical effect . this kids ' chair has a weight capacity of 150 lbs.""")

'559 - Furniture > Baby & Toddler Furniture > High Chairs & Booster Seats'

In [ ]:
set_taxonomy("""this is a new line of modern home-chef apron never fails to be bold and
beautiful . this apron is perfect for the aspiring chefs in your home ages 4-8.
the fabric is perfectly weighted denim , that looks and feels amazingly light
and durable , with all the advantages of cotton .""")

'639 - Home & Garden > Kitchen & Dining > Tableware > Aprons'

In [ ]:
set_taxonomy("""sweet dreams are a breeze when your little one is snuggled up in wildkin toddler
sheet set ! both the top sheet and fitted sheet fit a standard toddler mattress
, making this set perfect for transitioning from crib to big kid bed . the
toddler sheet set is made from 100 % cotton with a 205 thread count , making it
perfect for cozy cuddles . this lightweight material makes it breathable enough
for hot summer nights while keeping your child nice and warm during colder
months . each toddler sheet set features vibrant , playful designs that
coordinate with another wildkin room décor . from comforters to duvet covers ,
to storage cubes , your child will love seeing their favorite designs pop up
throughout their room .""")

'2314 - Home & Garden > Linens & Bedding > Bedding > Bed Sheets'

In [ ]:
product_ids = []
taxs = []

import textwrap

def wrap_text(text, width=80):
    if text is None:
        return text
    if not isinstance(text, str):
        return ''
    wrapped_text = textwrap.fill(text, width=width)
    return wrapped_text


counter = 0
for idx, row in downsample.loc[labeled_products].iterrows():
    product_name = row['product_name']
    if not isinstance(product_name, str):
        product_name = ""
    product_description = row['product_description']
    if not isinstance(product_description, str):
        product_description = ""
    tag = set_taxonomy(row['product_description'])
    if (counter % 10) == 0:
        print("--")
        print(row['product_name'])
        print(wrap_text(row['product_description']))
        print(counter, len(labeled_products), tag)
    counter += 1
    taxs.append(tag)
    product_ids.append(idx)

Streaming output truncated to the last 5000 lines.
400 6034 6356 - Furniture > Cabinets & Storage
--
zolia solid wood 3 drawer end table
our functional sideboard exudes a vintage charm and will make a distinctive
addition to your home . the cabinet is of mango wood which is solid , stable ,
durable , and beautiful , and the craftsmanship adds to its spectacular retro
style . every step of the process is carried out with the greatest care , be it
polishing , painting , or lacquering . the craftsmanship and beautiful wood
grains make every piece of furniture unique and slightly different from the next
. designed with 3 spacious drawers , this sideboard keeps your magazines , books
, dvds , and other essentials well organized and within reach .
410 6034 447 - Furniture > Cabinets & Storage > Buffets & Sideboards
--
nicosia tufted upholstered standard bed
add some majesty to your mornings , starting with this upholstered bed . full of
victorian-inspired charm , the fleur-de-lis motif , dia

In [ ]:
llm_tax = pd.DataFrame({
    "product_id": product_ids,
    "llm_tax": taxs
}).groupby('product_id').first().reset_index()


# Save to pickle
llm_tax.to_pickle("/content/drive/MyDrive/wands-gar/llm_taxs_by_id.pkl")
llm_tax

,product_id,llm_tax
0,0,505764 - Furniture > Beds & Accessories > Beds...
1,5,4148 - Furniture > Cabinets & Storage > Vanities
2,6,4148 - Furniture > Cabinets & Storage > Vanities
3,7,2081 - Furniture > Cabinets & Storage > Vaniti...
4,12,693 - Home & Garden > Decor > Figurines
...,...,...
6029,42979,2081 - Furniture > Cabinets & Storage > Vaniti...
6030,42980,2081 - Furniture > Cabinets & Storage > Vaniti...
6031,42981,2081 - Furniture > Cabinets & Storage > Vaniti...
6032,42982,2081 - Furniture > Cabinets & Storage > Vaniti...


In [ ]:
llm_tax.iloc[0]['llm_tax']

'505764 - Furniture > Beds & Accessories > Beds & Bed Frames'

In [ ]:
downsample_w_llm = downsample.copy()
downsample_w_llm['llm_tax'] = llm_tax.set_index('product_id')
downsample_w_llm['llm_tax'].fillna('', inplace=True)
downsample_w_llm

,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,product_name_snowball,product_description_snowball,product_class_snowball,product_name_ws,product_description_ws,product_class_ws,llm_tax
product_id,,,,,,,,,,,,,,,
0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0,"Terms({'solid', 'wood', 'bed', 'platform'})","Terms({'nightstand', 'style', 'our', 'relax', ...",Terms({'bed'}),"Terms({'solid', 'wood', 'bed', 'platform'})","Terms({'comfortably', 'nightstand', 'style', '...",Terms({'beds'}),505764 - Furniture > Beds & Accessories > Beds...
1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0,"Terms({'clad', 'cooker', 'qt', 'slow', 'all', ...","Terms({'when', 'entertain', 's', 'a', 'at', 't...","Terms({'cooker', 'slow'})","Terms({'clad', 'cooker', 'qt', 'slow', 'all', ...","Terms({'when', 'seamlessly', 'everything', 's'...","Terms({'slow', 'cookers'})",
2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0,"Terms({'clad', '6', 'cooker', '5', 'electr', '...","Terms({'essenti', 'and', 'a', 'meal', 'on', 't...","Terms({'cooker', 'slow'})","Terms({'clad', 'electrics', '6', 'cooker', '5'...","Terms({'and', 'essential', 'a', 'on', 'this', ...","Terms({'slow', 'cookers'})",
3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0,"Terms({'clad', 'pizza', 'profession', 'all', '...","Terms({'and', 'the', 'was', 'blade', 'compleme...","Terms({'slicer', 'and', 'grater', 'peeler'})","Terms({'clad', 'pizza', 'professional', 'tools...","Terms({'rotary', 'and', 'the', 'was', 'blade',...","Terms({'peelers', 'and', 'slicers', 'graters'})",
4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0,"Terms({'prestig', 'alcott', 'with', 'round', '...","Terms({'which', 'inspir', 'entri', 'an', 'and'...","Terms({'knob', 'door'})","Terms({'alcott', 'passage', 'with', 'round', '...","Terms({'which', 'instantly', 'privacy', 'an', ...","Terms({'door', 'knobs'})",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0,"Terms({'fix', 'panel', 'balanc', 'head', 'pres...","Terms({'turn', 'an', 'and', 'bodi', 'the', 'be...","Terms({'shower', 'panel'})","Terms({'balanced', 'panel', 'fixed', 'head', '...","Terms({'beautiful', 'turn', 'an', 'and', 'the'...","Terms({'panels', 'shower'})",
42990,emmeline 5 piece breakfast dining set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,None,basematerialdetails : steel| : gray wood|ofhar...,1314.0,4.5,864.0,"Terms({'emmelin', 'piec', 'breakfast', 'dine',...",Terms(set()),"Terms({'tabl', 'set', 'dine'})","Terms({'breakfast', '5', 'dining', 'piece', 'e...",Terms(set()),"Terms({'dining', 'table', 'sets'})",
42991,maloney 3 piece pub table set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,this pub table set includes 1 counter height t...,additionaltoolsrequirednotincluded : power dri...,49.0,4.0,41.0,"Terms({'tabl', '3', 'piec', 'maloney', 'pub', ...","Terms({'kitchen', 'bistro', 'compact', '

## Construct a Taxonomy Similarity Search

In [ ]:

def path_tokenizer(tax):
    nodes = tax.split('>')
    paths = []
    if len(nodes) == 0:
        return ''
    curr_path = nodes[0]
    paths = [curr_path]
    for node in nodes[1:]:
        curr_path = curr_path + ' > ' + node.strip().lower()
        paths.append(curr_path)
    return paths

path_tokenizer('505764 - Furniture > Beds & Accessories > Beds & Bed Frames')

['505764 - Furniture ',
 '505764 - Furniture  > beds & accessories',
 '505764 - Furniture  > beds & accessories > beds & bed frames']

In [ ]:
downsample_w_llm['llm_tax_path'] = SearchArray.index(downsample_w_llm['llm_tax'],
                                                     tokenizer=path_tokenizer)

2024-09-03 13:44:29,079 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2024-09-03 13:44:29,091 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2024-09-03 13:44:29,096 - searcharray.indexing - INFO - Tokenizing 42986 documents


...

INFO:searcharray.indexing:Batch tokenization complete


2024-09-03 13:44:30,143 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2024-09-03 13:44:30,161 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


In [ ]:
def search_tax(query_tax, downsample, N=10):
    scores = np.zeros(len(downsample))
    for path in path_tokenizer(query_tax):
        scores += downsample['llm_tax_path'].array.score(path)

    sorted_idx = np.argsort(scores)[::-1][:N]
    results = downsample.iloc[sorted_idx]
    results['score'] = scores[sorted_idx]

    return results

results = search_tax('6499 - Furniture > Chairs',
                      downsample_w_llm)

results[['product_name', 'product_description', 'llm_tax', 'score']]

<ipython-input-42-07e25f90ff86>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['score'] = scores[sorted_idx]


,product_name,product_description,llm_tax,score
product_id,,,,
17417,traditional kids recliner,this traditional child recliner is great for r...,"6499 - Furniture > Chairs > Arm Chairs, Reclin...",4.478587
22762,ulmer kids cotton sleeper,black/white sleeper/sofa .,"6499 - Furniture > Chairs > Arm Chairs, Reclin...",4.478587
42514,forman kids recliner with cup holder,let your little one lay back on movie night or...,"6499 - Furniture > Chairs > Arm Chairs, Reclin...",4.478587
31671,eneida reclining 2 piece living room set,"proudly made in american , selecting from choc...","6499 - Furniture > Chairs > Arm Chairs, Reclin...",4.478587
33368,oss modern kids chair,give your kiddo 's bedroom or play area the pe...,"6499 - Furniture > Chairs > Arm Chairs, Reclin...",4.478587
654,modern l-shaped reversible sectional sofa couc...,this sectional sofa is the perfect way to add ...,"6499 - Furniture > Chairs > Arm Chairs, Reclin...",4.478587
14338,kids chair with cup holder,"do n't forget about the kiddos , babies includ...","6499 - Furniture > Chairs > Arm Chairs, Reclin...",4.478587
6419,coomes kids recliner,add a lovely touch to the decor with this prod...,"6499 - Furniture > Chairs > Arm Chairs, Reclin...",4.478587
22766,eresus kids chair,this exciting addition to zoomie kids new `` e...,"6499 - Furniture > Chairs > Arm Chairs, Reclin...",4.478587


### Taxonomy for each test query

In [ ]:
def set_taxonomy_q(query):

    if not query:
        return ""

    test = f"""
    Product Description: {query}
    """
    # print("API CALL!")

    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
        "role": "system",
        "content": prompt
        },
        {
        "role": "user",
        "content": test
        }
    ],
    temperature=0.1,
    max_tokens=256,
    top_p=1
    )
    resp = response.choices[0].message.content
    resp_split = resp.split("\n")
    title, tags, desc = "", "", ""
    label = "Taxonomy:"
    for i in range(0, len(resp_split)):
        if resp_split[i].startswith(label):
            tags = resp_split[i][len(label)+1:]

    return tags

query_tax_dict = {}

for query in test_queries:
    query_tax = set_taxonomy_q(query)
    if 'No appropriate classification' not in query_tax:
        query_tax_dict[query] = query_tax
    print(query, query_tax)

#     'desk for kids': '4191 - Furniture > Office Furniture > Desks',
#     'jordanna solid wood rocking': '2002 - Furniture > Chairs > Rocking Chairs',
#     'oriental vanity': '4148 - Furniture > Cabinets & Storage > Vanities',
#     'nectar queen mattress': '2696 - Furniture > Beds & Accessories > Mattresses',
#     'bedroom wall decor floral, multicolored with some teal (prints)': '500044 - Home & Garden > Decor > Artwork > Posters, Prints, & Visual Artwork',
#    # '48 in entry table with side by side drawer': '465 - Furniture > Shelving > Bookcases & Standing Shelves'
#     'alyse 8 light': '2249 - Home & Garden > Lighting > Lighting Fixtures > Chandeliers',
#     'kari 2 piece': 'Furniture > Furniture Sets > Living Room Furniture Sets',
#     'tufted upholstered bed diamond':'505764 - Furniture > Beds & Accessories > Beds & Bed Frames'
# }

desk for kids 4191 - Furniture > Office Furniture > Desks
jordanna solid wood rocking 2002 - Furniture > Chairs > Rocking Chairs
oriental vanity 4148 - Furniture > Cabinets & Storage > Vanities
nectar queen mattress 2696 - Furniture > Beds & Accessories > Mattresses
bedroom wall decor floral, multicolored with some teal (prints) 500044 - Home & Garden > Decor > Artwork > Posters, Prints, & Visual Artwork
48 in entry table with side by side drawer 465 - Furniture > Shelving > Bookcases & Standing Shelves
alyse 8 light 2249 - Home & Garden > Lighting > Lighting Fixtures > Chandeliers
kari 2 piece 6346 - Furniture > Furniture Sets > Living Room Furniture Sets
tufted upholstered bed diamond 505764 - Furniture > Beds & Accessories > Beds & Bed Frames


In [ ]:
manual_tax = {
    'desk for kids': '4191 - Furniture > Office Furniture > Desks',
    'jordanna solid wood rocking': '2002 - Furniture > Chairs > Rocking Chairs',
    'oriental vanity': '4148 - Furniture > Cabinets & Storage > Vanities',
    'nectar queen mattress': '2696 - Furniture > Beds & Accessories > Mattresses',
    'bedroom wall decor floral, multicolored with some teal (prints)': '500044 - Home & Garden > Decor > Artwork > Posters, Prints, & Visual Artwork',
   # '48 in entry table with side by side drawer': '465 - Furniture > Shelving > Bookcases & Standing Shelves'
    'alyse 8 light': '2249 - Home & Garden > Lighting > Lighting Fixtures > Chandeliers',
    'kari 2 piece': 'Furniture > Furniture Sets > Living Room Furniture Sets',
    'tufted upholstered bed diamond':'505764 - Furniture > Beds & Accessories > Beds & Bed Frames'
}

def search_tax_boost2(query,
                      downsample,
                      labels,
                      fields=["product_name_snowball", "product_description_snowball"],
                      N=10):
    debug = {}
    scores = np.zeros(len(downsample))
    terms_so_far = set()
    for term in snowball_tokenizer(query):
        if term in terms_so_far:
            continue
        terms_so_far.add(term)
        best_term_match = np.zeros(len(downsample))
        for field in fields:
            field_w_boost = field.split('^')
            if len(field_w_boost) == 1:
                boost = 1
                search_field = field
            elif len(field_w_boost) == 2:
                boost = int(field_w_boost[1])
                search_field = field_w_boost[0]

            term_score = downsample[search_field].array.score(term) * boost
            best_term_match = np.maximum(best_term_match, term_score)
            debug[(term, field)] = term_score

        scores += best_term_match

    try:
        query_tax = manual_tax[query]
        for path in path_tokenizer(query_tax):
            print(path)
            query_tax_score = downsample['llm_tax_path'].array.score(path)
            debug[(path, 'llm_tax_path')] = query_tax_score
            scores += query_tax_score
    except KeyError:
        pass

    query_labels = labels[labels['query'] == query]
    results = labeled_results(scores, query_labels, downsample, debug=debug, N=N)
    print(results.columns)
    return results

def search_all(labels, queries, downsample, search_fn):
    all_results = []

    for query in queries:
        results = search_fn(query, downsample, labels)
        results['rank'] = np.arange(len(results)) + 1
        results['query'] = query
        all_results.append(results)

    return pd.concat(all_results)

def search_llm2(query,
               downsample,
               labels):
    return search_tax_boost2(query, downsample, labels, fields=["product_name_snowball", "product_description_snowball"])

results_test2 = search_all(labels, test_queries,
                          downsample_w_llm, search_llm2)
results_test2

4191 - Furniture 
4191 - Furniture  > office furniture
4191 - Furniture  > office furniture > desks
Index(['product_id', 'product_name', 'product_class', 'category hierarchy',
       'product_description', 'product_features', 'rating_count',
       'average_rating', 'review_count', 'product_name_snowball',
       'product_description_snowball', 'product_class_snowball',
       'product_name_ws', 'product_description_ws', 'product_class_ws',
       'llm_tax', 'llm_tax_path', 'debug_product_name_snowball_desk',
       'debug_product_description_snowball_desk',
       'debug_product_name_snowball_for',
       'debug_product_description_snowball_for',
       'debug_product_name_snowball_kid',
       'debug_product_description_snowball_kid',
       'debug_llm_tax_path_4191 - Furniture ',
       'debug_llm_tax_path_4191 - Furniture  > office furniture',
       'debug_llm_tax_path_4191 - Furniture  > office furniture > desks',
       'debug', 'scores', 'grade'],
      dtype='object')
2002 - F

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,product_name_snowball,...,debug_product_description_snowball_tuft,debug_product_name_snowball_upholst,debug_product_description_snowball_upholst,debug_product_name_snowball_bed,debug_product_description_snowball_bed,debug_product_name_snowball_diamond,debug_product_description_snowball_diamond,debug_llm_tax_path_505764 - Furniture,debug_llm_tax_path_505764 - Furniture > beds & accessories,debug_llm_tax_path_505764 - Furniture > beds & accessories > beds & bed frames
0,19145,osterley 31.5 '' w writing desk and chair set,Kids Desks,Baby & Kids / Toddler & Kids Bedroom Furniture...,height adjustable wooden student desk and chai...,overallproductweight:49.6|overallwidth-sidetos...,2.0,4.0,2.0,"Terms({'write', 'w', 'desk', '5', 'and', 'oste...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25007,monarch hill haven kids 47.48 '' w writing des...,Kids Desks,Sale / Closeout / Toddler & Kids Bedroom Furni...,it ’ s what every intrepid explorer dreams of ...,estimatedtimetosetup:120|additionaltoolsrequir...,80.0,4.5,65.0,"Terms({'write', 'hill', 'w', '48', 'kid', 'wit...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9611,sesame street kids desk chair with storage com...,Kids Chairs|Licensed Products,Baby & Kids / Toddler & Kids Playroom / Playro...,add a functional appeal to your kid ’ s room w...,seatheight-floortoseat:10|productcare : wipe c...,344.0,5.0,259.0,"Terms({'sesam', 'desk', 'kid', 'and', 'street'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19211,"isabelle & max™ kids desk and chair set , heig...",None,Baby & Kids / Toddler & Kids Bedroom Furniture...,isabelle & max™ kids desk and chair set,chairframefinish : white|levelofassembly : ful...,21.0,4.0,20.0,"Terms({'metal', 'and', 'layer', 'larg', 'heigh...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21667,joppatowne 48 '' writing desk with hutch,Kids Desks,Baby & Kids / Toddler & Kids Bedroom Furniture...,there 's no reason for a kid 's room to be a m...,desktopwidth-sidetoside:48|desktopdepth-frontt...,NaN,NaN,NaN,"Terms({'write', 'hutch', '48', 'joppatown', 'w...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,14834,abbot diamond queen upholstered standard bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,manifest the extraordinary in your bedroom dec...,warrantylength:1 year|overallproductweight:82|...,90.0,4.5,63.0,"Terms({'bed', 'upholst', 'diamond', 'queen', '...",...,1.522166,1.554309,1.467250,1.383881,1.632850,3.163621,3.048794,1.168706,1.168706,1.168706
6,33376,melonie tufted upholstered standard bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,from its arch headboard to the elegant diamond...,overallwidth-sidetoside:78|overallheight-topto...,NaN,NaN,NaN,"Terms({'meloni', 'bed', 'standard', 'upholst',...",...,2.136845,1.661958,1.536990,1.479726,1.257240,0.000000,2.347469,1.168706,1.168706,1.168706
7,40512,acamar upholstered standard bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,this wingback bed sets a modern feel with its ...,overallheight-toptobottom:56|weightcapacity:50...,115.0,4.5,91.0,"Terms({'bed', 'standard', 'acamar', 'upholst'})",...,1.522166,1.785627,1.996178,1.589835,1.855856,0.000000,2.240954,1.168706,1.168706,1.168706
8,8360,rita tufted upholstered low profile standard bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,a wingback design pair with diamond-tufted det...,headboarddepth-fronttoback:10|overalllength-he...,144.0,4.5,90.0,"Terms({'rita', 'bed', 'profil', 'low', 'standa...",...,1.748894,1.459757,0.000000,1.299697,1.790775,0.000000,2.574747,1.168706,1.168706,1.168706


In [ ]:
def ndcg_m(results):
    max_dcg = np.sum((2**(np.ones(10) + 1)) / np.log(np.arange(10) + 2))
    results['gain'] = (2**results['grade'] - 1) / np.log(results['rank'] + 1)
    dcgs = results.groupby('query')['gain'].sum()
    dcgs = dcgs.sort_values()
    return dcgs / max_dcg

ndcg_ms_baseline = ndcg_m(results_baseline)

ndcg_ms_test2 = ndcg_m(results_test2)
ndcg_ms_test2 = ndcg_ms_test2[test_queries]
ndcg_ms_test2.mean()

0.3558915078832073

### Queries improved / harmed

In [ ]:
delta_ndcg = ndcg_ms_test2 - ndcg_ms_baseline
delta_ndcg.sort_values()[test_queries]

,gain
query,
desk for kids,0.231159
jordanna solid wood rocking,-0.055023
oriental vanity,0.000000
nectar queen mattress,0.000000
"bedroom wall decor floral, multicolored with some teal (prints)",0.113597
48 in entry table with side by side drawer,0.000000
alyse 8 light,0.000000
kari 2 piece,0.000000
tufted upholstered bed diamond,0.000000


In [ ]:
import textwrap
import json

def wrap_text(text, width=80):
    if text is None:
        return text
    if not isinstance(text, str):
        return ''
    wrapped_text = textwrap.fill(text, width=width)
    return wrapped_text


def dump_results(results, query):
    print("********")
    print(query)
    print("********")
    cols = ['rank', 'grade', 'product_name', 'product_id', 'scores', 'product_description']
    if 'llm_title' in results.columns:
        cols.append('llm_title')
        cols.append('llm_description')
    for idx, row in results.loc[results['query'] == query].iterrows():
        print("--")
        print("RANK:", row['rank'])
        print("grade:", row['grade'])
        print("product_id:", row['product_id'])
        print("product_name:", row['product_name'])
        print(wrap_text(row['product_description']))
        if 'llm_tax' in row:
            print("LLM TAX:", row['llm_tax'])
        if 'llm_title' in row:
            print()
            print("LLM:")
            print(row['llm_title'])
            print(wrap_text(row['llm_description']))
            print()
        print('score:', row['scores'])
        print("grade:", row['grade'])

        if 'debug' in row:
            dbg = json.dumps(row['debug'], indent=2)
            print(dbg)

dump_results(results_test2,
             'bedroom wall decor floral, multicolored with some teal (prints)')

********
bedroom wall decor floral, multicolored with some teal (prints)
********
--
RANK: 1
grade: 0.0
product_id: 801
product_name: roosters - floral - unframed print on paper
paintbrush stroked feathers decorate the bodies of these modern roosters , and
fun patterns make them stand out in style . eli halpin 's wonderful collection
of nature wall art is uniquely elegant . add some flair to your decor with her
colorful and unforgettable designs .
LLM TAX: 500044 - Home & Garden > Decor > Artwork > Posters, Prints, & Visual Artwork
score: 15.607175305485725
grade: 0.0
{
  "debug_product_name_snowball_bedroom": 0.0,
  "debug_product_description_snowball_bedroom": 0.0,
  "debug_product_name_snowball_wall": 0.0,
  "debug_product_description_snowball_wall": 1.3229438066482544,
  "debug_product_name_snowball_decor": 0.0,
  "debug_product_description_snowball_decor": 1.2734286785125732,
  "debug_product_name_snowball_floral": 2.14264178276062,
  "debug_product_description_snowball_floral": 0

## Add cleaned titles too

Is cleaning titles additive?

[Run previous notebook first](https://colab.research.google.com/drive/1jgnnzHU7RT-lQp_5JqoHmWDH2QFhtdth?)

In [ ]:
llm_titles_fixed = pd.read_pickle("/content/drive/MyDrive/wands-gar/llm_tags_by_id.pkl").set_index('product_id')

downsample_w_llm['llm_title_fixed'] = llm_titles_fixed['llm_title_fixed']
downsample_w_llm['llm_title_fixed'].fillna('', inplace=True)

downsample_w_llm['title_concat'] = downsample_w_llm['product_name'] + ' ' + downsample_w_llm['llm_title_fixed']

downsample_w_llm['llm_title_fixed_snowball'] = SearchArray.index(downsample_w_llm['llm_title_fixed'],
                                                                 tokenizer=snowball_tokenizer)

downsample_w_llm['llm_title_concat_snowball'] = SearchArray.index(downsample_w_llm['title_concat'],
                                                                  tokenizer=snowball_tokenizer)

2024-09-03 13:45:05,516 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2024-09-03 13:45:05,527 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2024-09-03 13:45:05,531 - searcharray.indexing - INFO - Tokenizing 42986 documents


...

INFO:searcharray.indexing:Batch tokenization complete


2024-09-03 13:45:08,260 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2024-09-03 13:45:08,296 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


In [ ]:
def search_llm3(query,
               downsample,
               labels):
    return search_tax_boost2(query, downsample, labels, fields=["llm_title_concat_snowball", "product_description_snowball"])

results_test3 = search_all(labels, test_queries,
                          downsample_w_llm, search_llm3)
results_test3

4191 - Furniture 
4191 - Furniture  > office furniture
4191 - Furniture  > office furniture > desks
Index(['product_id', 'product_name', 'product_class', 'category hierarchy',
       'product_description', 'product_features', 'rating_count',
       'average_rating', 'review_count', 'product_name_snowball',
       'product_description_snowball', 'product_class_snowball',
       'product_name_ws', 'product_description_ws', 'product_class_ws',
       'llm_tax', 'llm_tax_path', 'llm_title_fixed', 'title_concat',
       'llm_title_fixed_snowball', 'llm_title_concat_snowball',
       'debug_llm_title_concat_snowball_desk',
       'debug_product_description_snowball_desk',
       'debug_llm_title_concat_snowball_for',
       'debug_product_description_snowball_for',
       'debug_llm_title_concat_snowball_kid',
       'debug_product_description_snowball_kid',
       'debug_llm_tax_path_4191 - Furniture ',
       'debug_llm_tax_path_4191 - Furniture  > office furniture',
       'debug_llm_tax_

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,product_name_snowball,...,debug_product_description_snowball_tuft,debug_llm_title_concat_snowball_upholst,debug_product_description_snowball_upholst,debug_llm_title_concat_snowball_bed,debug_product_description_snowball_bed,debug_llm_title_concat_snowball_diamond,debug_product_description_snowball_diamond,debug_llm_tax_path_505764 - Furniture,debug_llm_tax_path_505764 - Furniture > beds & accessories,debug_llm_tax_path_505764 - Furniture > beds & accessories > beds & bed frames
0,19145,osterley 31.5 '' w writing desk and chair set,Kids Desks,Baby & Kids / Toddler & Kids Bedroom Furniture...,height adjustable wooden student desk and chai...,overallproductweight:49.6|overallwidth-sidetos...,2.0,4.0,2.0,"Terms({'write', 'w', 'desk', '5', 'and', 'oste...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17410,poulan kids study desk,Kids Desks,Baby & Kids / Toddler & Kids Bedroom Furniture...,give your little one a spot to power through p...,topcolor : warm cherry|basecolor : willliamsbu...,9.0,5.0,9.0,"Terms({'poulan', 'desk', 'studi', 'kid'})",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23916,willison kids study desk,Kids Desks,Baby & Kids / Toddler & Kids Bedroom Furniture...,the wrought studio kid 's complete desk system...,desktopwidth-sidetoside:42.91|agegroup:7 to 8 ...,32.0,5.0,19.0,"Terms({'desk', 'studi', 'willison', 'kid'})",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38525,aadhya kids study desk,Kids Desks,Baby & Kids / Toddler & Kids Bedroom Furniture...,curate a cozy and organized workspace for your...,basecolor : white|dsprimaryproductstyle : yout...,107.0,4.0,86.0,"Terms({'aadhya', 'desk', 'studi', 'kid'})",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,25007,monarch hill haven kids 47.48 '' w writing des...,Kids Desks,Sale / Closeout / Toddler & Kids Bedroom Furni...,it ’ s what every intrepid explorer dreams of ...,estimatedtimetosetup:120|additionaltoolsrequir...,80.0,4.5,65.0,"Terms({'write', 'hill', 'w', '48', 'kid', 'wit...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,33376,melonie tufted upholstered standard bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,from its arch headboard to the elegant diamond...,overallwidth-sidetoside:78|overallheight-topto...,NaN,NaN,NaN,"Terms({'meloni', 'bed', 'standard', 'upholst',...",...,2.136845,1.900766,1.536990,1.698050,1.257240,0.000000,2.347469,1.168706,1.168706,1.168706
6,40373,demontfort king upholstered platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,set the scene for serene relaxation with this ...,weightcapacity:800|levelofassembly : partial a...,NaN,NaN,NaN,"Terms({'bed', 'platform', 'demontfort', 'king'...",...,1.686107,1.974186,1.625277,1.763640,1.329458,0.000000,2.482311,1.168706,1.168706,1.168706
7,17765,full/double upholstered low profile sleigh bed,None,Furniture / Bedroom Furniture / Beds & Headboa...,this bed is both beautiful and simple . with a...,overalllength-headtotoe:79.9|color : beige|ove...,NaN,NaN,NaN,"Terms({'bed', 'profil', 'low', 'upholst', 'ful...",...,2.002417,1.270681,0.000000,1.637165,1.578861,0.000000,2.947987,1.168706,1.168706,1.168706
8,37936,kachig king upholstered platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"the bed is a striking , contemporary bed . the...",weightcapacity:440|dssecondaryproductstyle : t...,NaN,NaN,NaN,"Terms({'bed', 'platform', 'king', 'upholst', '...",...,1.639033,1.411000,1.579901,1.763640,1.716086,2.826933,2.413007,1.168706,1.168706,1.168706


In [ ]:
def ndcg_m(results):
    max_dcg = np.sum((2**(np.ones(10) + 1)) / np.log(np.arange(10) + 2))
    results['gain'] = (2**results['grade'] - 1) / np.log(results['rank'] + 1)
    dcgs = results.groupby('query')['gain'].sum()
    dcgs = dcgs.sort_values()
    return dcgs / max_dcg

ndcg_ms_test3 = ndcg_m(results_test3)
ndcg_ms_test3 = ndcg_ms_test3[test_queries]
ndcg_ms_test3.mean()

0.3689556300982193

In [ ]:
ndcg_ms_test3 - ndcg_ms_baseline

,gain
query,
48 in entry table with side by side drawer,-0.032749
alyse 8 light,0.000000
"bedroom wall decor floral, multicolored with some teal (prints)",0.113695
desk for kids,0.310413
jordanna solid wood rocking,-0.075330
kari 2 piece,0.000000
nectar queen mattress,0.000000
oriental vanity,0.000000
tufted upholstered bed diamond,0.091282


## Compare to best possible

In [ ]:
best_results = labels[labels['query'].isin(test_queries)].sort_values(['query', 'grade'],
                                           ascending=[True, False]).groupby('query_id').head(10).merge(downsample, how='left', on='product_id')

best_results['rank'] = best_results.groupby('query_id').cumcount() + 1
best_results

,id,query_id,product_id,label,grade,query,query_class,product_name,product_class,category hierarchy,...,rating_count,average_rating,review_count,product_name_snowball,product_description_snowball,product_class_snowball,product_name_ws,product_description_ws,product_class_ws,rank
0,13194,101,1860,Exact,2.0,48 in entry table with side by side drawer,Sofa & Console Tables,stronghurst 48 '' solid wood console table,Desks|Sofa & Console Tables,Furniture / Living Room Furniture / Console Ta...,...,448.0,4.5,319.0,"Terms({'tabl', 'solid', '48', 'stronghurst', '...","Terms({'while', 'openwork', 'finish', 'you', '...","Terms({'sofa', 'consol', 'desk', 'tabl'})","Terms({'solid', '48', 'stronghurst', 'wood', '...","Terms({'while', 'baskets', 'openwork', 'finish...","Terms({'tables', 'sofa', 'desks', 'console'})",1
1,13195,101,32892,Exact,2.0,48 in entry table with side by side drawer,Sofa & Console Tables,stroud metal console table,Sofa & Console Tables,Furniture / Living Room Furniture / Console Ta...,...,256.0,4.5,153.0,"Terms({'tabl', 'consol', 'metal', 'stroud'})","Terms({'while', 'metal', 'one', 'sever', 'fini...","Terms({'sofa', 'consol', 'tabl'})","Terms({'stroud', 'metal', 'table', 'console'})","Terms({'while', 'metal', 'one', 'finish', 'gre...","Terms({'tables', 'sofa', 'console'})",2
2,75900,101,32907,Exact,2.0,48 in entry table with side by side drawer,Sofa & Console Tables,bautista 48 '' console table,Sofa & Console Tables,Furniture / Living Room Furniture / Console Ta...,...,209.0,4.5,131.0,"Terms({'tabl', 'consol', 'bautista', '48'})","Terms({'metal', 'yet', 'and', 'style', 'the', ...","Terms({'sofa', 'consol', 'tabl'})","Terms({'bautista', 'console', 'table', '48'})","Terms({'metal', 'house', 'yet', 'and', 'style'...","Terms({'tables', 'sofa', 'console'})",3
3,75921,101,26500,Exact,2.0,48 in entry table with side by side drawer,Sofa & Console Tables,trinidad 48 '' console table,Sofa & Console Tables,Furniture / Living Room Furniture / Console Ta...,...,461.0,4.5,283.0,"Terms({'tabl', 'consol', 'trinidad', '48'})","Terms({'while', 'metal', 'finish', 'great', 'y...","Terms({'sofa', 'consol', 'tabl'})","Terms({'trinidad', 'console', 'table', '48'})","Terms({'while', 'metal', 'finish', 'great', 'y...","Terms({'tables', 'sofa', 'console'})",4
4,13070,101,700,Partial,1.0,48 in entry table with side by side drawer,Sofa & Console Tables,2 - drawer end table,End Tables,Furniture / Living Room Furniture / Coffee Tab...,...,2.0,2.5,2.0,"Terms({'tabl', 'end', 'drawer', '2'})",Terms(set()),"Terms({'tabl', 'end'})","Terms({'drawer', 'end', 'table', '2'})",Terms(set()),"Terms({'tables', 'end'})",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,12964,100,42880,Exact,2.0,tufted upholstered bed diamond,Beds,agnese tufted upholstered low profile platform...,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,...,1048.0,5.0,721.0,"Terms({'bed', 'platform', 'profil', 'low', 'up...","Terms({'slat', 'realli', 'need', 's', 'a', 're...",Terms({'bed'}),"Terms({'bed', 'profile', 'tufted', 'platform',...","Terms({'headaches', 'slat', 'need', 'confidenc...",Terms({'beds'}),6
86,12965,100,17735,Exact,2.0,tufted upholstered bed diamond,Beds,aileen tufted upholstered low profile platform...,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,...,668.0,4.5,437.0,"Terms({'bed', 'platform', 'profil', 'low', 'up...","Terms({'fill', 'which', 'trim', 'headboard', '...",Terms({'bed'}),"Terms({'bed', 'profile', 'tufted', 'platform',...","Terms({'which', 'trim', 'nailhead', 'provide',...",Terms({'beds'}),7
87,12968,100,40521,Exact,2.0,tufted upholstered bed diamond,Beds,alcantara diamond tufted upholstered standard bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,...,2047.0,4.5,1445.0,"Terms({'bed', 'standard', 'upholst', 'diamond'...","Terms({'fill', 'dress', 't', 'don', 'and', 'mo...",Terms({'bed'}),"Terms({'bed', 'tufted', 'upholstered', 'diamon...","Terms({'t', 'don', 'showing', ""'"", 'and', 'mos...

In [ ]:
ndcg_ms_ideal = ndcg_m(best_results)
ndcg_ms_ideal.mean(), ndcg_ms_ideal

(0.4774549302739514,
 query
 nectar queen mattress                                              0.250000
 oriental vanity                                                    0.250000
 alyse 8 light                                                      0.360046
 jordanna solid wood rocking                                        0.360046
 kari 2 piece                                                       0.360046
 48 in entry table with side by side drawer                         0.531894
 bedroom wall decor floral, multicolored with some teal (prints)    0.685062
 desk for kids                                                      0.750000
 tufted upholstered bed diamond                                     0.750000
 Name: gain, dtype: float64)

In [ ]:
(ndcg_ms_test3 / ndcg_ms_ideal).sort_values()

,gain
query,
"bedroom wall decor floral, multicolored with some teal (prints)",0.304359
kari 2 piece,0.554886
48 in entry table with side by side drawer,0.628088
jordanna solid wood rocking,0.790776
tufted upholstered bed diamond,0.911299
alyse 8 light,1.000000
desk for kids,1.000000
nectar queen mattress,1.000000
oriental vanity,1.000000


In [ ]:
results_test3[results_test3['query'] == 'bedroom wall decor floral, multicolored with some teal (prints)'][['scores', 'grade', 'product_id', 'rank', 'product_name',
                                                                                                            'product_description']]

,scores,grade,product_id,rank,product_name,product_description
0,15.017978,0.0,801,1,roosters - floral - unframed print on paper,paintbrush stroked feathers decorate the bodie...
1,13.135927,1.0,926,2,morning iris - 2 piece picture frame graphic a...,"break up that boring empty wall , and add some..."
2,12.814586,1.0,4471,3,wood framed wall decor with fern fronds - 6 pi...,empty entryway wall ? some spare space in the ...
3,12.485875,1.0,31232,4,floral and botanical paint beautiful growth fl...,make any space in your home innovative and uni...
4,12.473040,1.0,224,5,sunflower decorative spoon and fork wall decor...,the flower and leave are embossed with excelle...
5,12.446437,0.0,9163,6,orchids floral arrangement in decorative vase,add a touch of nature to your decor with these...
6,12.018021,1.0,4168,7,succulent - wrapped canvas print,empty entryway wall ? some spare space in the ...
7,11.933303,1.0,25045,8,translucent tulips ii sq aqua crop - 2 piece p...,from setting the aesthetic of a room to showca...
8,11.774593,2.0,4649,9,painted petals xix by tristan scott - wrapped ...,the painted petals xix by tristan scott - wrap...
9,11.435450,1.0,37647,10,leaf wall decor,this leaf wall decor features gold finished sa...


In [ ]:
best_results[best_results['query'] == 'bedroom wall decor floral, multicolored with some teal (prints)']

,id,query_id,product_id,label,grade,query,query_class,product_name,product_class,category hierarchy,...,average_rating,review_count,product_name_snowball,product_description_snowball,product_class_snowball,product_name_ws,product_description_ws,product_class_ws,rank,gain
20,26197,188,6733,Exact,2.0,"bedroom wall decor floral, multicolored with s...",Wall Décor,abstract floral 1 by design fabrikken - wrappe...,Wall Art,Décor & Pillows / Art / All Wall Art / Pink Wa...,...,NaN,NaN,"Terms({'wrap', 'print', 'floral', 'by', '1', '...","Terms({'style', 'eco', 'need', 'a', 'piec', 't...","Terms({'wall', 'art'})","Terms({'print', 'wrapped', 'floral', 'by', '1'...","Terms({'style', 'eco', 'need', 'a', 'immense',...","Terms({'wall', 'art'})",1,4.328085
21,26201,188,11156,Exact,2.0,"bedroom wall decor floral, multicolored with s...",Wall Décor,flowers in unity teal by david bromstad - pict...,Wall Art,Décor & Pillows / Art / All Wall Art,...,NaN,NaN,"Terms({'flower', 'print', 'in', 'frame', 'unit...","Terms({'and', 'will', 'of', 'sure', 'piec', 't...","Terms({'wall', 'art'})","Terms({'unity', 'print', 'in', 'frame', 'by', ...","Terms({'nature', 'and', 'piece', 'will', 'flow...","Terms({'wall', 'art'})",2,2.730718
22,26202,188,12177,Exact,2.0,"bedroom wall decor floral, multicolored with s...",Wall Décor,full bloom x floral by danhui nai - graphic ar...,Wall Art,Décor & Pillows / Art / All Wall Art / Red Wal...,...,NaN,NaN,"Terms({'print', 'graphic', 'nai', 'floral', 'b...","Terms({'vibrant', 'decad', 'great', 'remain', ...","Terms({'wall', 'art'})","Terms({'print', 'graphic', 'nai', 'floral', 'b...","Terms({'vibrant', 'exceptional', 'great', 'str...","Terms({'wall', 'art'})",3,2.164043
23,26203,188,12178,Exact,2.0,"bedroom wall decor floral, multicolored with s...",Wall Décor,full bloom xiii floral by danhui nai - graphic...,Wall Art,Décor & Pillows / Art / All Wall Art / Red Wal...,...,NaN,NaN,"Terms({'xiii', 'print', 'graphic', 'nai', 'flo...","Terms({'vibrant', 'decad', 'great', 'remain', ...","Terms({'wall', 'art'})","Terms({'xiii', 'print', 'graphic', 'nai', 'flo...","Terms({'vibrant', 'exceptional', 'great', 'str...","Terms({'wall', 'art'})",4,1.864005
24,26206,188,37582,Exact,2.0,"bedroom wall decor floral, multicolored with s...",Wall Décor,painted petals iv-b by tristan scott - 4 piece...,Wall Art,Décor & Pillows / Art / All Wall Art / Green W...,...,4.5,18.0,"Terms({'wrap', 'piec', 'print', 'scott', 'peta...","Terms({'and', 'style', 'offic', 'across', 'whi...","Terms({'wall', 'art'})","Terms({'print', 'wrapped', 'scott', 'by', 'b',...","Terms({'gallery', 'and', 'style', 'across', 'w...","Terms({'wall', 'art'})",5,1.674332
25,26207,188,11645,Exact,2.0,"bedroom wall decor floral, multicolored with s...",Wall Décor,painted petals lxi by tristan scott - 2 piece ...,Wall Art,Décor & Pillows / Art / All Wall Art,...,4.5,133.0,"Terms({'wrap', 'piec', 'print', 'graphic', 'sc...","Terms({'convers', 'showcas', 'vibrant', 'pink'...","Terms({'wall', 'art'})","Terms({'print', 'wrapped', 'graphic', 'scott',...","Terms({'mounting', 'vibrant', 'pink', 'canvase...","Terms({'wall', 'art'})",6,1.541695
26,26208,188,4649,Exact,2.0,"bedroom wall decor floral, multicolored with s...",Wall Décor,painted petals xix by tristan scott - wrapped ...,Wall Art,Décor & Pillows / Art / All Wall Art / Green W...,...,4.5,46.0,"Terms({'wrap', 'print', 'graphic', 'scott', 'p...","Terms({'scott', 'and', 'style', 'by', 'tristan...","Terms({'wall', 'art'})","Terms({'print', 'wrapped', 'graphic', 'scott',...","Terms({'scott', 'gallery', 'and', 'style', 'by...","Terms({'wall', 'art'})",7,1.442695
27,26209,188,41680,Exact,2.0,"bedroom wall decor floral, multicolored with s...",Wall Décor,sea isle wildflowers - advertisements print on...,Wall Art,Décor & Pillows / Art / All Wall Art,...,5.0,856.0,"Terms({'sea', 'print', 'isl', 'canva', 'advert...","Terms({'the', 'will', 'use', 'beauti', 'a', 'g...","Terms({'wall', 'art'})","Terms({'sea', 'print', 'advertisements', 'wild